In [ ]:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import csv
import pickle

import sys
sys.path.append('.../')

import dynTimeWarp

from mpl_toolkits.mplot3d import Axes3D
#import dynTimeWarp
from scipy import interpolate
from scipy.signal import savgol_filter
from scipy.signal import argrelextrema
from itertools import combinations

In [ ]:
'''the columns names in the csv files always have either a space in the beginning or '\t\t' therefore renaming them
Further, the values for the Vive data are saved as strings and with a comma therefore changing these into floats and with a dot
To do: extend preprocessing to all variables? (at least to those of interest that may come later)'''

def preprocessing(df):
    df = df.rename(columns={
    ' Trial':'Trial', '\t\tEvent':'Event', ' TrialType':'TrialType',
    '\t\tZeit':'Zeit', ' Sensorgroesse':'Sensorgroesse', ' \t\tBall_x':'Ball_x', ' Ball_y':'Ball_z', ' Ball_z':'Ball_y', 
    ' \t\tController_x':'Controller_x', ' Controller_y':'Controller_z', ' Controller_z':'Controller_y', 
    ' \t\tpointTracker017_x':'pointTracker017_x', ' pointTracker017_y':'pointTracker17_z', ' pointTracker017_z':'pointTracker17_y', 
    ' \t\tpointTracker023_x':'pointTracker023_x', ' pointTracker023_y':'pointTracker023_z', ' pointTracker023_z':'pointTracker023_y', 
    ' \t\tpointTracker024_x':'pointTracker024_x', ' pointTracker024_y':'pointTracker024_z', ' pointTracker024_z':'pointTracker024_y',
    ' \t\tlinkable40All_x':'linkable40All_x', ' linkable40All_y':'linkable40All_z', ' linkable40All_z':'linkable40All_y',
    ' \t\tViveTracker_x':'ViveTracker_x', ' ViveTracker_y':'ViveTracker_z', ' ViveTracker_z':'ViveTracker_y'
    })

    df["Sensorgroesse"] = df["Sensorgroesse"].astype(str).str.replace(',','.')
    df["Ball_x"] = df["Ball_x"].astype(str).str.replace(',','.')
    df["Ball_y"] = df["Ball_y"].astype(str).str.replace(',','.')
    df["Ball_z"] = df["Ball_z"].astype(str).str.replace(',','.')
    df["Controller_x"] = df["Controller_x"].astype(str).str.replace(',','.')
    df["Controller_y"] = df["Controller_y"].astype(str).str.replace(',','.')
    df["Controller_z"] = df["Controller_z"].astype(str).str.replace(',','.')

    df["Ball_x"] = df["Ball_x"].astype(float)
    df["Ball_y"] = df["Ball_y"].astype(float)
    df["Ball_z"] = df["Ball_z"].astype(float)

    df["Controller_x"] = df["Controller_x"].astype(float)
    df["Controller_y"] = df["Controller_y"].astype(float)
    df["Controller_z"] = df["Controller_z"].astype(float)

    #df["ViveTracker_x"] = df["ViveTracker_x"].astype(str).str.replace(',','.')
    #df["ViveTracker_y"] = df["ViveTracker_y"].astype(str).str.replace(',','.')
    #df["ViveTracker_z"] = df["ViveTracker_z"].astype(str).str.replace(',','.')

    #df["ViveTracker_x"] = df["ViveTracker_x"].astype(float)
    #df["ViveTracker_y"] = df["ViveTracker_y"].astype(float)
    #df["ViveTracker_z"] = df["ViveTracker_z"].astype(float)

    

    df.loc[df['Event'].astype(str).str.contains('ball placed at startshelf', na=False), 'Event'] = 'ball placed at startshelf'

    

    return df

In [ ]:
def df_column_switch(df, column1, column2):
    i = list(df.columns)
    a, b = i.index(column1), i.index(column2)
    i[b], i[a] = i[a], i[b]
    df = df[i]
    return df

In [ ]:
'''function to get a specific trial within a block
input: the number of the desired trial, the block of which we want to get the trial
output: dataframe that contains all entries of every column for that trial'''

def makeTrial(number, Block):
    Trial = Block.loc[Block['Trial']==number] 

    return Trial 

In [ ]:
'''make the conditions. Save all trials of each condition in this block to one corresponding variable. 
then, collect all conditions in a tuple (or better list?)'''

def getConditions(Block):
    NoPert = Block.loc[Block['TrialType']=='B']
    Pert125 = Block.loc[(Block['Sensorgroesse']=='0.125000') & (Block['TrialType']=='A')]
    Pert150 = Block.loc[(Block['Sensorgroesse']=='0.150000') & (Block['TrialType']=='A')]
    Pert175 = Block.loc[(Block['Sensorgroesse']=='0.175000') & (Block['TrialType']=='A')]
    Pert200 = Block.loc[(Block['Sensorgroesse']=='0.200000') & (Block['TrialType']=='A')]
    Pert225 = Block.loc[(Block['Sensorgroesse']=='0.225000') & (Block['TrialType']=='A')]
    Pert250 = Block.loc[(Block['Sensorgroesse']=='0.250000') & (Block['TrialType']=='A')]

    Conditions = (NoPert,Pert125,Pert150,Pert175,Pert200,Pert225,Pert250 )

    return Conditions

In [ ]:
'''to get the trial numbers of each condition, we create an empty list, loop through the conditions in "Conditions" and collect the trial numbers of each condition in that list
output: a list with one entry for each condition where each entry contains the trial numbers of that condition'''

def getTrialNumbers(Conditions):
    CondTrialNumbers = []
    
    for i in range (len(Conditions)):
        CondTrialNumbers.append(pd.unique(Conditions[i]['Trial']))

    return CondTrialNumbers

In [ ]:
'''create an empty list, loop through the variable that contains the trial numbers for each condition and collect each trial of a specific condition in that list
output: a list that contains the trials of a specific condition'''

def makeTrials(CondTrialNumbers, Index, Block):
    CondTrials = []
    for i in range (len(CondTrialNumbers[Index])):
        CondTrials.append(makeTrial(CondTrialNumbers[Index][i], Block))

    return CondTrials

In [ ]:
'''for further processing, we set the row index for every trial based on the events happening'''

def setEventIndex(condTrials):
    for trial in range(len(condTrials)):
        condTrials[trial] = condTrials[trial].set_index('Event',drop=False)
    
    return condTrials

In [ ]:
'''checking whether a trial was a success i.e. the ball has been successfully grabbed and collect booleans for success / no succes in one list for each condition'''

def isItSuccess(condTrials):
    SuccessList = []
    for trial in range(len(condTrials)):
        SuccessList.append(condTrials[trial]['Event'].str.contains('ball has been grabbed').any())
    
    return SuccessList

In [ ]:
'''get the indices of all "True"'s, that is all trials where the ball has been grabbed successfully'''

def SuccessIndices(SuccessList):
    SuccessIndices = []
    for i in range(len(SuccessList)):
        if SuccessList[i]==True:
            SuccessIndices.append(i+1)

    return SuccessIndices

In [ ]:
'''define the relevant movement for each trial from passing the startpoint to grabbing the ball and collect them in a s; one s for each condition again'''
 
def getMoves(condSuccessIndices, condTrials):
    condMoves = []
    for i in condSuccessIndices:
        condMoves.append(condTrials[i-1].loc["ball placed at startshelf":"ball has been grabbed",:])

    return condMoves

In [ ]:
def rescaleCoordinates(Move):
    rescaledControllerDFs = [] 
    for i in range(len(cond1ControllerDFs)): 
        rescaledControllerDFs.append((cond1ControllerDFs[i]['x'][:]-cond1ControllerDFs[0]['x'][0])*-1)


In [ ]:
def PertLeftRight(condMoveList):
    CondLeftList = []
    CondRightList = []
    for trial in range(len(condMoveList)):
        if condMoveList[trial]['Event'].str.contains('Perturbation left').any():
            CondLeftList.append(condMoveList[trial])
        elif condMoveList[trial]['Event'].str.contains('Perturbation right').any():
            CondRightList.append(condMoveList[trial])
        else: 
            pass
    
    return CondLeftList, CondRightList

In [ ]:
'''function to get the ball movements of each trial within a condition
input: condition of interest
output: a list for each axis of the ball movement containing the ball's movmeents for each trial in the condition of interest'''

def getBallMoves(cond):
    BallX = []
    BallY = []
    BallZ = []

    for trial in cond:
        BallX.append(trial['Ball_x'])
        BallY.append(trial['Ball_y'])
        BallZ.append(trial['Ball_z'])

    return BallX, BallY, BallZ

In [ ]:
def saveMoves2(Path, FileName, condMoveList):
        for i in range(len(condMoveList)):
                    condMoveList[i][list(condMoveList[i].loc[:,'pointTracker017_x':'pointTracker023_z']) + ['Zeit']].to_csv(Path+FileName+str(i+1)+'.txt', sep='\t', index=False, header = False)


In [ ]:
def SaveMovements(AllBlocks, pathArmMoves):
    for block in AllBlocks:
        B = preprocessing(block)
        BConds = getConditions(B)
        CondTrialNumbers = getTrialNumbers(BConds)

        Cond1Trials = makeTrials(CondTrialNumbers, 0, B)
        Cond2Trials = makeTrials(CondTrialNumbers, 1, B)
        Cond3Trials = makeTrials(CondTrialNumbers, 2, B)
        Cond4Trials = makeTrials(CondTrialNumbers, 3, B)
        Cond5Trials = makeTrials(CondTrialNumbers, 4, B)
        Cond6Trials = makeTrials(CondTrialNumbers, 5, B)
        Cond7Trials = makeTrials(CondTrialNumbers, 6, B)

        CondList = (Cond1Trials,Cond2Trials,Cond3Trials,Cond4Trials,Cond5Trials,Cond6Trials,Cond7Trials)

        for i in CondList:
            setEventIndex(i)

        cond1SuccessList = isItSuccess(Cond1Trials)
        cond2SuccessList = isItSuccess(Cond2Trials)
        cond3SuccessList = isItSuccess(Cond3Trials)
        cond4SuccessList = isItSuccess(Cond4Trials)
        cond5SuccessList = isItSuccess(Cond5Trials)
        cond6SuccessList = isItSuccess(Cond6Trials)
        cond7SuccessList = isItSuccess(Cond7Trials)

        cond1SuccessIndices = SuccessIndices(cond1SuccessList)
        cond2SuccessIndices = SuccessIndices(cond2SuccessList)
        cond3SuccessIndices = SuccessIndices(cond3SuccessList)
        cond4SuccessIndices = SuccessIndices(cond4SuccessList)
        cond5SuccessIndices = SuccessIndices(cond5SuccessList)
        cond6SuccessIndices = SuccessIndices(cond6SuccessList)
        cond7SuccessIndices = SuccessIndices(cond7SuccessList)

        cond1Moves = getMoves(cond1SuccessIndices, Cond1Trials)
        cond2Moves = getMoves(cond2SuccessIndices, Cond2Trials)
        cond3Moves = getMoves(cond3SuccessIndices, Cond3Trials)
        cond4Moves = getMoves(cond4SuccessIndices, Cond4Trials)
        cond5Moves = getMoves(cond5SuccessIndices, Cond5Trials)
        cond6Moves = getMoves(cond6SuccessIndices, Cond6Trials)
        cond7Moves = getMoves(cond7SuccessIndices, Cond7Trials)

        #cond1MovesLeft, cond1MovesRight = PertLeftRight(cond1Moves)
        cond2MovesLeft, cond2MovesRight = PertLeftRight(cond2Moves)
        cond3MovesLeft, cond3MovesRight = PertLeftRight(cond3Moves)
        cond4MovesLeft, cond4MovesRight = PertLeftRight(cond4Moves)
        cond5MovesLeft, cond5MovesRight = PertLeftRight(cond5Moves)
        cond6MovesLeft, cond6MovesRight = PertLeftRight(cond6Moves)
        cond7MovesLeft, cond7MovesRight = PertLeftRight(cond7Moves)
        
        Blocknumber = "B"+str(B.iloc[0,1])
        
        cond1Moves = saveMoves2(Path = pathArmMoves + 'NoPerturbation/', FileName = Blocknumber + 'NoPertMove', condMoveList = cond1Moves)

        cond2MovesLeft = saveMoves2(Path = pathArmMoves + 'Perturbation125/Left/',  FileName = Blocknumber + 'Pert125LeftMove', condMoveList = cond2MovesLeft)
        cond2MovesRight = saveMoves2(Path = pathArmMoves + 'Perturbation125/Right/', FileName = Blocknumber + 'Pert125RightMove', condMoveList = cond2MovesRight) 

        cond3MovesLeft = saveMoves2(Path = pathArmMoves + 'Perturbation150/Left/', FileName = Blocknumber + 'Pert150Move', condMoveList = cond3MovesLeft)
        cond3MovesRight = saveMoves2(Path = pathArmMoves + 'Perturbation150/Right/', FileName = Blocknumber + 'Pert150Move', condMoveList = cond3MovesRight) 
        
        cond4MovesLeft = saveMoves2(Path = pathArmMoves + 'Perturbation175/Left/',  FileName = Blocknumber + 'Pert175Move', condMoveList = cond4MovesLeft)
        cond4MovesRight = saveMoves2(Path = pathArmMoves + 'Perturbation175/Right/',  FileName = Blocknumber + 'Pert175Move', condMoveList = cond4MovesRight) 
        
        cond5MovesLeft = saveMoves2(Path = pathArmMoves + 'Perturbation200/Left/',  FileName = Blocknumber + 'Pert200Move', condMoveList = cond5MovesLeft)
        cond5MovesRight = saveMoves2(Path = pathArmMoves + 'Perturbation200/Right/',  FileName = Blocknumber + 'Pert200Move', condMoveList = cond5MovesRight) 
        
        cond6MovesLeft = saveMoves2(Path = pathArmMoves + 'Perturbation225/Left/',  FileName = Blocknumber + 'Pert225Move', condMoveList = cond6MovesLeft)
        cond6MovesRight = saveMoves2(Path = pathArmMoves + 'Perturbation225/Right/',  FileName = Blocknumber + 'Pert225Move', condMoveList = cond6MovesRight) 
        
        cond7MovesLeft = saveMoves2(Path = pathArmMoves + 'Perturbation250/Left/',  FileName = Blocknumber + 'Pert250Move', condMoveList = cond7MovesLeft)
        cond7MovesRight = saveMoves2(Path = pathArmMoves + 'Perturbation250/Right/', FileName = Blocknumber + 'Pert250Move', condMoveList = cond7MovesRight)



In [ ]:
def makeArmMoveFolders(participant):
    pathArmMoves = '.../'+participant+'.../'
    FolderNames = ('NoPerturbation/', 'Perturbation125/Left/', 'Perturbation125/Right/', 'Perturbation150/Left/', 'Perturbation150/Right/', 'Perturbation175/Left/', 'Perturbation175/Right/', 
    'Perturbation200/Left/', 'Perturbation200/Right/', 'Perturbation225/Left/', 'Perturbation225/Right/', 'Perturbation250/Left/', 'Perturbation250/Right/')
    
    for folderName in FolderNames:
        os.makedirs(pathArmMoves+folderName)



In [ ]:
def replaceBallMoveString(condMoves):
    for i in range(len(condMoves)):
        condMoves[i]['Event'].replace(to_replace = r'^entered ballMove.*$',value= 'entered ballMove', regex=True, inplace=True)
    return condMoves

In [ ]:
def perturbationOnsets(condMoves):
    
    moveUntilPert = []
    for i in range(len(condMoves)):
        moveUntilPert.append(condMoves[i].loc["ball placed at startshelf":"entered ballMove",'Zeit'])

    pertOnsetTimes = []
    for i in range(len(moveUntilPert)):
        pertOnsetTimes.append(moveUntilPert[i][-1] - moveUntilPert[i][0])
    
    return pertOnsetTimes, moveUntilPert

In [ ]:
def GetPertOnsets(AllBlocks):

    cond2LeftPertOnsets = []
    cond2RightPertOnsets = []

    cond3LeftPertOnsets = []
    cond3RightPertOnsets = []

    cond4LeftPertOnsets = []
    cond4RightPertOnsets = []

    cond5LeftPertOnsets = []
    cond5RightPertOnsets = []

    cond6LeftPertOnsets = []
    cond6RightPertOnsets = []

    cond7LeftPertOnsets = []
    cond7RightPertOnsets = []

    for block in AllBlocks:
        B = preprocessing(block)
        BConds = getConditions(B)
        CondTrialNumbers = getTrialNumbers(BConds)

        Cond1Trials = makeTrials(CondTrialNumbers, 0, B)
        Cond2Trials = makeTrials(CondTrialNumbers, 1, B)
        Cond3Trials = makeTrials(CondTrialNumbers, 2, B)
        Cond4Trials = makeTrials(CondTrialNumbers, 3, B)
        Cond5Trials = makeTrials(CondTrialNumbers, 4, B)
        Cond6Trials = makeTrials(CondTrialNumbers, 5, B)
        Cond7Trials = makeTrials(CondTrialNumbers, 6, B)

        CondList = (Cond1Trials,Cond2Trials,Cond3Trials,Cond4Trials,Cond5Trials,Cond6Trials,Cond7Trials)

        for i in CondList:
            setEventIndex(i)

        cond1SuccessList = isItSuccess(Cond1Trials)
        cond2SuccessList = isItSuccess(Cond2Trials)
        cond3SuccessList = isItSuccess(Cond3Trials)
        cond4SuccessList = isItSuccess(Cond4Trials)
        cond5SuccessList = isItSuccess(Cond5Trials)
        cond6SuccessList = isItSuccess(Cond6Trials)
        cond7SuccessList = isItSuccess(Cond7Trials)

        cond1SuccessIndices = SuccessIndices(cond1SuccessList)
        cond2SuccessIndices = SuccessIndices(cond2SuccessList)
        cond3SuccessIndices = SuccessIndices(cond3SuccessList)
        cond4SuccessIndices = SuccessIndices(cond4SuccessList)
        cond5SuccessIndices = SuccessIndices(cond5SuccessList)
        cond6SuccessIndices = SuccessIndices(cond6SuccessList)
        cond7SuccessIndices = SuccessIndices(cond7SuccessList)

        cond1Moves = getMoves(cond1SuccessIndices, Cond1Trials)
        cond2Moves = getMoves(cond2SuccessIndices, Cond2Trials)
        cond3Moves = getMoves(cond3SuccessIndices, Cond3Trials)
        cond4Moves = getMoves(cond4SuccessIndices, Cond4Trials)
        cond5Moves = getMoves(cond5SuccessIndices, Cond5Trials)
        cond6Moves = getMoves(cond6SuccessIndices, Cond6Trials)
        cond7Moves = getMoves(cond7SuccessIndices, Cond7Trials)

        cond2Moves = getMoves(cond2SuccessIndices, Cond2Trials)
        cond3Moves = getMoves(cond3SuccessIndices, Cond3Trials)
        cond4Moves = getMoves(cond4SuccessIndices, Cond4Trials)
        cond5Moves = getMoves(cond5SuccessIndices, Cond5Trials)
        cond6Moves = getMoves(cond6SuccessIndices, Cond6Trials)
        cond7Moves = getMoves(cond7SuccessIndices, Cond7Trials)

        cond2Moves = replaceBallMoveString(cond2Moves)
        cond3Moves = replaceBallMoveString(cond3Moves)
        cond4Moves = replaceBallMoveString(cond4Moves)
        cond5Moves = replaceBallMoveString(cond5Moves)
        cond6Moves = replaceBallMoveString(cond6Moves)
        cond7Moves = replaceBallMoveString(cond7Moves)

        #cond1MovesLeft, cond1MovesRight = PertLeftRight(cond1Moves)
        cond2MovesLeft, cond2MovesRight = PertLeftRight(cond2Moves)
        cond3MovesLeft, cond3MovesRight = PertLeftRight(cond3Moves)
        cond4MovesLeft, cond4MovesRight = PertLeftRight(cond4Moves)
        cond5MovesLeft, cond5MovesRight = PertLeftRight(cond5Moves)
        cond6MovesLeft, cond6MovesRight = PertLeftRight(cond6Moves)
        cond7MovesLeft, cond7MovesRight = PertLeftRight(cond7Moves)


        cond2LeftPertOnsets.append(perturbationOnsets(cond2MovesLeft))
        cond2RightPertOnsets.append(perturbationOnsets(cond2MovesRight))

        cond3LeftPertOnsets.append(perturbationOnsets(cond3MovesLeft))
        cond3RightPertOnsets.append(perturbationOnsets(cond3MovesRight))

        cond4LeftPertOnsets.append(perturbationOnsets(cond4MovesLeft))
        cond4RightPertOnsets.append(perturbationOnsets(cond4MovesRight))

        cond5LeftPertOnsets.append(perturbationOnsets(cond5MovesLeft))
        cond5RightPertOnsets.append(perturbationOnsets(cond5MovesRight))

        cond6LeftPertOnsets.append(perturbationOnsets(cond6MovesLeft))
        cond6RightPertOnsets.append(perturbationOnsets(cond6MovesRight))

        cond7LeftPertOnsets.append(perturbationOnsets(cond7MovesLeft))
        cond7RightPertOnsets.append(perturbationOnsets(cond7MovesRight))

    return (cond2LeftPertOnsets, cond2RightPertOnsets, cond3LeftPertOnsets, cond3RightPertOnsets, cond4LeftPertOnsets, cond4RightPertOnsets, cond5LeftPertOnsets, cond5RightPertOnsets,
        cond6LeftPertOnsets, cond6RightPertOnsets, cond7LeftPertOnsets, cond7RightPertOnsets)

In [ ]:
def getViveMoves(condMoveList, ViveX, ViveZ):
    ViveMoves = []
    for i in range(len(condMoveList)):
        ViveMoves.append(condMoveList[i][list(condMoveList[i].loc[:,ViveX:ViveZ]) + ['Zeit']])
        ViveMoves[i]

    return ViveMoves

In [ ]:
def getViveMovements(AllBlocks, ViveX, ViveZ):

    cond1ViveMoves = []
    cond2LeftViveMoves = []
    cond2RightViveMoves = []
    cond3LeftViveMoves = []
    cond3RightViveMoves = []
    cond4LeftViveMoves = []
    cond4RightViveMoves = []
    cond5LeftViveMoves = []
    cond5RightViveMoves = []
    cond6LeftViveMoves = []
    cond6RightViveMoves = []
    cond7LeftViveMoves = []
    cond7RightViveMoves = []

    for block in AllBlocks:
        B = preprocessing(block)
        BConds = getConditions(B)
        CondTrialNumbers = getTrialNumbers(BConds)

        Cond1Trials = makeTrials(CondTrialNumbers, 0, B)
        Cond2Trials = makeTrials(CondTrialNumbers, 1, B)
        Cond3Trials = makeTrials(CondTrialNumbers, 2, B)
        Cond4Trials = makeTrials(CondTrialNumbers, 3, B)
        Cond5Trials = makeTrials(CondTrialNumbers, 4, B)
        Cond6Trials = makeTrials(CondTrialNumbers, 5, B)
        Cond7Trials = makeTrials(CondTrialNumbers, 6, B)

        CondList = (Cond1Trials,Cond2Trials,Cond3Trials,Cond4Trials,Cond5Trials,Cond6Trials,Cond7Trials)

        for i in CondList:
            setEventIndex(i)

        cond1SuccessList = isItSuccess(Cond1Trials)
        cond2SuccessList = isItSuccess(Cond2Trials)
        cond3SuccessList = isItSuccess(Cond3Trials)
        cond4SuccessList = isItSuccess(Cond4Trials)
        cond5SuccessList = isItSuccess(Cond5Trials)
        cond6SuccessList = isItSuccess(Cond6Trials)
        cond7SuccessList = isItSuccess(Cond7Trials)

        cond1SuccessIndices = SuccessIndices(cond1SuccessList)
        cond2SuccessIndices = SuccessIndices(cond2SuccessList)
        cond3SuccessIndices = SuccessIndices(cond3SuccessList)
        cond4SuccessIndices = SuccessIndices(cond4SuccessList)
        cond5SuccessIndices = SuccessIndices(cond5SuccessList)
        cond6SuccessIndices = SuccessIndices(cond6SuccessList)
        cond7SuccessIndices = SuccessIndices(cond7SuccessList)

        cond1Moves = getMoves(cond1SuccessIndices, Cond1Trials)
        cond2Moves = getMoves(cond2SuccessIndices, Cond2Trials)
        cond3Moves = getMoves(cond3SuccessIndices, Cond3Trials)
        cond4Moves = getMoves(cond4SuccessIndices, Cond4Trials)
        cond5Moves = getMoves(cond5SuccessIndices, Cond5Trials)
        cond6Moves = getMoves(cond6SuccessIndices, Cond6Trials)
        cond7Moves = getMoves(cond7SuccessIndices, Cond7Trials)

        #cond1MovesLeft, cond1MovesRight = PertLeftRight(cond1Moves)
        cond2MovesLeft, cond2MovesRight = PertLeftRight(cond2Moves)
        cond3MovesLeft, cond3MovesRight = PertLeftRight(cond3Moves)
        cond4MovesLeft, cond4MovesRight = PertLeftRight(cond4Moves)
        cond5MovesLeft, cond5MovesRight = PertLeftRight(cond5Moves)
        cond6MovesLeft, cond6MovesRight = PertLeftRight(cond6Moves)
        cond7MovesLeft, cond7MovesRight = PertLeftRight(cond7Moves)
        
        Blocknumber = "B"+str(B.iloc[0,1])

        cond1ViveMoves.append(getViveMoves(cond1Moves, ViveX, ViveZ))
        cond2LeftViveMoves.append(getViveMoves(cond2MovesLeft, ViveX, ViveZ))
        cond2RightViveMoves.append(getViveMoves(cond2MovesRight, ViveX, ViveZ))
        cond3LeftViveMoves.append(getViveMoves(cond3MovesLeft, ViveX, ViveZ))
        cond3RightViveMoves.append(getViveMoves(cond3MovesRight, ViveX, ViveZ))
        cond4LeftViveMoves.append(getViveMoves(cond4MovesLeft, ViveX, ViveZ))
        cond4RightViveMoves.append(getViveMoves(cond4MovesRight, ViveX, ViveZ))
        cond5LeftViveMoves.append(getViveMoves(cond5MovesLeft, ViveX, ViveZ))
        cond5RightViveMoves.append(getViveMoves(cond5MovesRight, ViveX, ViveZ))
        cond6LeftViveMoves.append(getViveMoves(cond6MovesLeft, ViveX, ViveZ))
        cond6RightViveMoves.append(getViveMoves(cond6MovesRight, ViveX, ViveZ))
        cond7LeftViveMoves.append(getViveMoves(cond7MovesLeft, ViveX, ViveZ))
        cond7RightViveMoves.append(getViveMoves(cond7MovesRight, ViveX, ViveZ))

    return (cond1ViveMoves, cond2LeftViveMoves, cond2RightViveMoves, cond3LeftViveMoves, cond3RightViveMoves, cond4LeftViveMoves, cond4RightViveMoves, cond5LeftViveMoves, cond5RightViveMoves,
            cond6LeftViveMoves, cond6RightViveMoves, cond7LeftViveMoves, cond7RightViveMoves)

In [ ]:
def flattenList(nestedList):
    flatList = [item for sublist in nestedList for item in sublist]
    return flatList

In [ ]:
def MakeViveMoveArray(cond):
    for i in range(len(cond)):
         cond[i] = np.asarray(cond[i])

    return cond

In [ ]:
def getBallDistances(cond):
    distances = []
    for i in range(len(cond)):
        distances.append(cond[i][-1,2]-cond[i][0,2])
    
    return distances 

In [ ]:
def inverseTransform(TM,q):
    Rotation = TM[:3,:3]
    Translation = TM[:3,3]
    InverseRotation = np.linalg.inv(Rotation)
    B = InverseRotation
    d = -InverseRotation@Translation
    q = np.atleast_2d(q)
    assert q.shape[1] == 3, 'q has the wrong shape.'
    p = np.einsum('ik,...k->...i', B, q)+d

    return p

In [ ]:
def getTMatrix(T):

    T = np.array(T)
    if T.shape != (1,16):
        T = T[0]
    else:
        pass
    TM = T.reshape(4,4)
    TMt = TM.T
    
    return TMt

In [ ]:
def transformViveCoordinates(cond):
    for i in range(len(cond)):
        cond[i] = np.delete(cond[i], 3, 1)
        cond[i] = inverseTransform(TMt, cond[i])

    return cond

In [ ]:
def makeResultFolders(P = '.../'):

    ResultFolderNames = ('MovementDurations/', 'Successes/', 'BallTrajectories/','PertCorrectionOnsets')

    for resultFolder in ResultFolderNames:
        os.makedirs(path + resultFolder)

In [ ]:
def plotRadiusEndBallMeans(condStringLeft, condStringRight, ballStart, distanceTravelledLeft, distanceTravelledRight, frames, color):
    meanTrajectory = pd.read_csv(meansPath + condStringLeft, header= None)
    meanTrajectory2 = pd.read_csv(meansPath + condStringRight, header= None) 

    meanTrajectory = np.array(meanTrajectory)
    meanTrajectory2 = np.array(meanTrajectory2)

    BallDistance = np.linspace(ballStart, ballStart+distanceTravelledLeft, frames)
    BallDistance2 = np.linspace(ballStart, ballStart+distanceTravelledRight, frames)

    plt.plot(meanTrajectory[:,0],meanTrajectory[:,2], color)
    plt.plot(cond3BallMovesLeft[0][:,0]*100, BallDistance*100)

    plt.plot(meanTrajectory2[:,0],meanTrajectory2[:,2], color)
    plt.plot(cond3BallMovesLeft[0][:,0]*100, BallDistance2*100)

In [ ]:
def collectTimes(cond):
    Times = []
    for i in range(len(cond)):
        Times.append(cond[i][:,3])
    
    return Times

In [ ]:
def makeControllerDF(cond, times):
    for i in range(len(cond)):
        cond[i]= np.insert(cond[i], 3, times[i], axis=1)

    controllerDFs = []

    for i in range(len(cond)):
        controllerDFs.append(pd.DataFrame(cond[i], columns=['x', 'z', 'y', 'Time']))

    return controllerDFs

In [ ]:
def pickleControllerMoves(fileName,condDFs):

    outfile = open(fileName, 'wb')

    pickle.dump(condDFs, outfile)

    outfile.close()

In [ ]:
def plotControllerMeans(condStringLeft, condStringRight, ballStart, distanceTravelledLeft, distanceTravelledRight, frames, color, label):
    meanTrajectory = pd.read_csv(meansPath + condStringLeft, header= None)
    meanTrajectory2 = pd.read_csv(meansPath + condStringRight, header= None) 

    meanTrajectory = np.array(meanTrajectory)
    meanTrajectory2 = np.array(meanTrajectory2)

    BallDistance = np.linspace(ballStart, ballStart+distanceTravelledLeft, frames)
    BallDistance2 = np.linspace(ballStart, ballStart+distanceTravelledRight, frames)

    plt.plot(meanTrajectory[:,0]*100,meanTrajectory[:,2]*100, color = color, label = label)
    plt.plot(cond3BallMovesLeft[0][:,0]*100, BallDistance*100)

    plt.plot(meanTrajectory2[:,0]*100,meanTrajectory2[:,2]*100, color)
    plt.plot(cond3BallMovesLeft[0][:,0]*100, BallDistance2*100)
    plt.legend()

In [ ]:
def getMoveDurations(cond):
    Durations = []
    for i in range(len(cond)):
        Durations.append(cond[i][-1,3] - cond[i][0,3])
    
    return Durations


In [ ]:
def getSuccessRates(AllBlocks):

    cond2LeftTrials = []
    cond2RightTrials = []
    cond3LeftTrials = []
    cond3RightTrials = []
    cond4LeftTrials = []
    cond4RightTrials = []
    cond5LeftTrials = []
    cond5RightTrials = []
    cond6LeftTrials = []
    cond6RightTrials = []
    cond7LeftTrials = []
    cond7RightTrials = []


    cond2LeftSuccesses = []
    cond2RightSuccesses = []
    cond3LeftSuccesses = []
    cond3RightSuccesses = []
    cond4LeftSuccesses = []
    cond4RightSuccesses = []
    cond5LeftSuccesses = []
    cond5RightSuccesses = []
    cond6LeftSuccesses = []
    cond6RightSuccesses = []
    cond7LeftSuccesses = []
    cond7RightSuccesses = []

    for block in AllBlocks:
        B = preprocessing(block)
        BConds = getConditions(B)
        CondTrialNumbers = getTrialNumbers(BConds)

        Cond1Trials = makeTrials(CondTrialNumbers, 0, B)
        Cond2Trials = makeTrials(CondTrialNumbers, 1, B)
        Cond3Trials = makeTrials(CondTrialNumbers, 2, B)
        Cond4Trials = makeTrials(CondTrialNumbers, 3, B)
        Cond5Trials = makeTrials(CondTrialNumbers, 4, B)
        Cond6Trials = makeTrials(CondTrialNumbers, 5, B)
        Cond7Trials = makeTrials(CondTrialNumbers, 6, B)

        CondList = (Cond1Trials,Cond2Trials,Cond3Trials,Cond4Trials,Cond5Trials,Cond6Trials,Cond7Trials)

        for i in CondList:
            setEventIndex(i)


        cond2MovesLeftAll, cond2MovesRightAll = PertLeftRight(Cond2Trials)
        cond2LeftTrials.append(len(cond2MovesLeftAll))
        cond2RightTrials.append(len(cond2MovesRightAll))

        cond3MovesLeftAll, cond3MovesRightAll = PertLeftRight(Cond3Trials)
        cond3LeftTrials.append(len(cond3MovesLeftAll))
        cond3RightTrials.append(len(cond3MovesRightAll))

        cond4MovesLeftAll, cond4MovesRightAll = PertLeftRight(Cond4Trials)
        cond4LeftTrials.append(len(cond4MovesLeftAll))
        cond4RightTrials.append(len(cond4MovesRightAll))

        cond5MovesLeftAll, cond5MovesRightAll = PertLeftRight(Cond5Trials)
        cond5LeftTrials.append(len(cond5MovesLeftAll))
        cond5RightTrials.append(len(cond5MovesRightAll))

        cond6MovesLeftAll, cond6MovesRightAll = PertLeftRight(Cond6Trials)
        cond6LeftTrials.append(len(cond6MovesLeftAll))
        cond6RightTrials.append(len(cond6MovesRightAll))

        cond7MovesLeftAll, cond7MovesRightAll = PertLeftRight(Cond7Trials)
        cond7LeftTrials.append(len(cond7MovesLeftAll))
        cond7RightTrials.append(len(cond7MovesRightAll))

        cond1SuccessList = isItSuccess(Cond1Trials)
        cond2SuccessList = isItSuccess(Cond2Trials)
        cond3SuccessList = isItSuccess(Cond3Trials)
        cond4SuccessList = isItSuccess(Cond4Trials)
        cond5SuccessList = isItSuccess(Cond5Trials)
        cond6SuccessList = isItSuccess(Cond6Trials)
        cond7SuccessList = isItSuccess(Cond7Trials)

        cond1SuccessIndices = SuccessIndices(cond1SuccessList)
        cond2SuccessIndices = SuccessIndices(cond2SuccessList)
        cond3SuccessIndices = SuccessIndices(cond3SuccessList)
        cond4SuccessIndices = SuccessIndices(cond4SuccessList)
        cond5SuccessIndices = SuccessIndices(cond5SuccessList)
        cond6SuccessIndices = SuccessIndices(cond6SuccessList)
        cond7SuccessIndices = SuccessIndices(cond7SuccessList)

        cond1Moves = getMoves(cond1SuccessIndices, Cond1Trials)
        cond2Moves = getMoves(cond2SuccessIndices, Cond2Trials)
        cond3Moves = getMoves(cond3SuccessIndices, Cond3Trials)
        cond4Moves = getMoves(cond4SuccessIndices, Cond4Trials)
        cond5Moves = getMoves(cond5SuccessIndices, Cond5Trials)
        cond6Moves = getMoves(cond6SuccessIndices, Cond6Trials)
        cond7Moves = getMoves(cond7SuccessIndices, Cond7Trials)

        #cond1MovesLeft, cond1MovesRight = PertLeftRight(cond1Moves)
        cond2MovesLeftSuccesses, cond2MovesRightSuccesses = PertLeftRight(cond2Moves)
        cond3MovesLeftSuccesses, cond3MovesRightSuccesses = PertLeftRight(cond3Moves)
        cond4MovesLeftSuccesses, cond4MovesRightSuccesses = PertLeftRight(cond4Moves)
        cond5MovesLeftSuccesses, cond5MovesRightSuccesses = PertLeftRight(cond5Moves)
        cond6MovesLeftSuccesses, cond6MovesRightSuccesses = PertLeftRight(cond6Moves)
        cond7MovesLeftSuccesses, cond7MovesRightSuccesses = PertLeftRight(cond7Moves)

        cond2LeftSuccesses.append(len(cond2MovesLeftSuccesses))
        cond2RightSuccesses.append(len(cond2MovesRightSuccesses))
        cond3LeftSuccesses.append(len(cond3MovesLeftSuccesses))
        cond3RightSuccesses.append(len(cond3MovesRightSuccesses))
        cond4LeftSuccesses.append(len(cond4MovesLeftSuccesses))
        cond4RightSuccesses.append(len(cond4MovesRightSuccesses))
        cond5LeftSuccesses.append(len(cond5MovesLeftSuccesses))
        cond5RightSuccesses.append(len(cond5MovesRightSuccesses))
        cond6LeftSuccesses.append(len(cond6MovesLeftSuccesses))
        cond6RightSuccesses.append(len(cond6MovesRightSuccesses))
        cond7LeftSuccesses.append(len(cond7MovesLeftSuccesses))
        cond7RightSuccesses.append(len(cond7MovesRightSuccesses))


    return (cond2LeftTrials, cond2RightTrials, cond3LeftTrials, cond3RightTrials, cond4LeftTrials, cond4RightTrials, cond5LeftTrials, cond5RightTrials, cond6LeftTrials, cond6RightTrials, cond7LeftTrials, cond7RightTrials, 
    cond2LeftSuccesses, cond2RightSuccesses, cond3LeftSuccesses, cond3RightSuccesses, cond4LeftSuccesses, cond4RightSuccesses, cond5LeftSuccesses, cond5RightSuccesses, cond6LeftSuccesses, cond6RightSuccesses, cond7LeftSuccesses, cond7RightSuccesses)

In [ ]:
'''function for calculating the overall success proportions per condition
input: number of success within a condition, number of trials within a condition
output: success proportion of the input condition'''

def successRatePerCondition(condSuccesses, condTrials):
    nSuccesses = sum(condSuccesses)
    nTrials = sum(condTrials)
    proportion = nSuccesses/nTrials

    return [nTrials, nSuccesses, proportion]

In [ ]:
'''function for calculating the overall success proportions for each block
outputs: list with number of trials of each block, list with number of successful trials in each block and list with overall success proportions in each block'''

def successRatePerBlock():
    trialsPerBlock = []
    successesPerBlock = []
    proportions = []
    for i in range(len(cond2TrialsLeft)):
        trialsPerBlock.append(cond2TrialsLeft[i]+cond2TrialsRight[i]+cond3TrialsLeft[i]+cond3TrialsRight[i]+cond4TrialsLeft[i]+cond4TrialsRight[i]+cond5TrialsLeft[i]+cond5TrialsRight[i]+cond6TrialsLeft[i]+cond6TrialsRight[i]+cond7TrialsLeft[i]+cond7TrialsRight[i])
        successesPerBlock.append(cond2LeftSuccesses[i]+cond2RightSuccesses[i]+cond3LeftSuccesses[i]+cond3RightSuccesses[i]+cond4LeftSuccesses[i]+cond4RightSuccesses[i]+cond5LeftSuccesses[i]+cond5RightSuccesses[i]+cond6LeftSuccesses[i]+cond6RightSuccesses[i]+cond7LeftSuccesses[i]+cond7RightSuccesses[i])

        proportions.append(successesPerBlock[i]/trialsPerBlock[i])

    return trialsPerBlock, successesPerBlock, proportions

In [ ]:
'''function to calculate the success proportions for each condition per block
inputs: number of successes within a condition, number of trials within a condition
output: list with success proportions within the input condition for each block'''

def successRatePerCondPerBlock(condSuccesses, condTrials):
    condProportions = []
    for i in range(len(cond2TrialsLeft)):
        condProportions.append((condSuccesses[i]/condTrials[i]) if condTrials[i]!= 0 else None)
    
    return condProportions

In [ ]:
def saveSuccessDF(SuccessesPath):
    columnNames = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'nTrials', 'nSuccesses', 'SuccessProportion']
    rowNames = ['cond2Left', 'cond2Right', 'cond3Left', 'cond3Right', 'cond4Left', 'cond4Right', 'cond5Left', 'cond5Right', 'cond6Left', 'cond6Right', 'cond7Left', 'cond7Right', 'nTrials', 'nSuccesses', 'SuccessProportion']
    d = np.array([[cond2LeftPropsPerBlock, cond2LeftSuccessProportion], [cond2RightPropsPerBlock, cond2RightSuccessProportion],
    [cond3LeftPropsPerBlock, cond3LeftSuccessProportion], [cond3RightPropsPerBlock, cond3RightSuccessProportion],
    [cond4LeftPropsPerBlock, cond4LeftSuccessProportion], [cond4RightPropsPerBlock, cond4RightSuccessProportion],
    [cond5LeftPropsPerBlock, cond5LeftSuccessProportion], [cond5RightPropsPerBlock, cond5RightSuccessProportion],
    [cond6LeftPropsPerBlock, cond6LeftSuccessProportion], [cond6RightPropsPerBlock, cond6RightSuccessProportion],
    [cond7LeftPropsPerBlock, cond7LeftSuccessProportion], [cond7RightPropsPerBlock, cond7RightSuccessProportion],
    [nTrialsPerBlock], [nSuccessesPerBlock], [proportionsPerBlock]])

    d = ([d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7], d[8], d[9], d[10], d[11], d[12], d[13], d[14]])

    for i in range(len(d)):
        d[i]=[j for sub in d[i] for j in sub]

    df = pd.DataFrame(d, columns = columnNames, index=rowNames)

    df['nTrials'] = df['nTrials'].astype('Int64')
    df['nSuccesses'] = df['nSuccesses'].astype('Int64')

    df.to_pickle(path = SuccessesPath+'/SuccessTable.pkl')


    

In [ ]:
def makeAllFolders(P = '.../'):

    ResultFolderNames = ('Trajectories/', 'WarpResults/', 'InterpolResults/', 'MeanTrajectories/')

    CondFolderNames = ('NoPerturbation/', 'Perturbation125/Left/', 'Perturbation125/Right/', 'Perturbation150/Left/', 'Perturbation150/Right/', 'Perturbation175/Left/', 'Perturbation175/Right/', 
    'Perturbation200/Left/', 'Perturbation200/Right/', 'Perturbation225/Left/', 'Perturbation225/Right/', 'Perturbation250/Left/', 'Perturbation250/Right/')
    
    JointFolderNames = ('Clavicle', 'Humerus', 'Radius', 'RadiusEnd')

    for resultFolder in ResultFolderNames:
        os.makedirs(path + resultFolder)

    for resultFolder in ResultFolderNames:
        for condFolder in CondFolderNames:
            os.makedirs(path + resultFolder + condFolder)

    for resultFolder in ResultFolderNames:
        for condFolder in CondFolderNames:
            for jointFolder in JointFolderNames:
                os.makedirs(path + resultFolder + condFolder + jointFolder)

In [ ]:
def makeControllerFolders(P = '.../'):
    ResultFolderNames = ('Trajectories/', 'WarpResults/', 'InterpolResults/', 'MeanTrajectories/')

    CondFolderNames = ('NoPerturbation/', 'Perturbation125/Left/', 'Perturbation125/Right/', 'Perturbation150/Left/', 'Perturbation150/Right/', 'Perturbation175/Left/', 'Perturbation175/Right/', 
    'Perturbation200/Left/', 'Perturbation200/Right/', 'Perturbation225/Left/', 'Perturbation225/Right/', 'Perturbation250/Left/', 'Perturbation250/Right/')

    ControllerFolderName = ('Controller',)

    for resultFolder in ResultFolderNames:
        for condFolder in CondFolderNames:
            for controllerFolder in ControllerFolderName:
                os.makedirs(path + resultFolder + condFolder + controllerFolder)

In [ ]:
def getMovements(condSorted):
    Moves = []
    for i in range (0, len(condSorted)):
        Moves.append(df.loc[(condSorted.iloc[i,1]//20):(condSorted.iloc[i,2]//20),:])

    return Moves


In [ ]:
def getArmMoves(Moves):

    for i in range (len(Moves)):
        Moves[i] = Moves[i][list(Moves[i].loc[:,'right_clavicle.X':'right_radiusEnd.Z'])+ ['Time']]
        ArmMoves = Moves
    return ArmMoves

#ArmMoves = getArmMoves(Moves)

In [ ]:
def getJoint(ArmMoves, X, Z):
    for i in range(len(ArmMoves)):
        ArmMoves[i] = ArmMoves[i][list(ArmMoves[i].loc[:,X:Z]) + ['Time']]
    Joint = ArmMoves
    Joint = list(sorted(Joint, key = len, reverse=True))
    return Joint

In [ ]:
def plotJointTrajectory(Joint,TrajectoryFolder):
    for i in range (len(Joint)):

        traj = np.array(Joint[i].iloc[:,0:3])
        #print(len(observed))

        t = np.array(Joint[i].loc[:,'Time'])
        tRange = t[-1]-t[0]
        dt = (t[1]-t[0])
        t = np.linspace(0, tRange, len(Joint[i].loc[:,'Time']))
        
        plt.clf()
        fig, axs = plt.subplots(3)
        axs[0].plot(t,traj[:,0]*-1)
        axs[1].plot(t,traj[:,1])
        axs[2].plot(t,traj[:,2])
        #plt.plot(traj[:,0]*-1, traj[:,2])
        


        plt.savefig(TrajectoryFolder+'Trajectory'+str(i)+".png")  


In [ ]:
def removeBadTrajectory(Joint, BadTrajectory):
    
    unwanted = BadTrajectory #list of indices with bad trajectories

    for ele in sorted(unwanted, reverse = True):
        del Joint[ele]
    print(len(Joint))
    return Joint

In [ ]:
def getMedianDuration(Joint):
    Times = []
    for i in range (len(Joint)):
        Times.append(len(Joint[i].Time))
    
    meanDuration = np.mean(Times)
    medianDuration = round(np.median(Times))

    # find value in times that is closest to median duration
    Times = np.asarray(Times)
    idx = (np.abs(Times-medianDuration)).argmin()


    MedianIndices = []

    for i in range(0,len(Times)):
        if Times[i]==Times[idx]:
            MedianIndices.append(i)
            print(MedianIndices)

    Medians = []

    for j in range (0, len(MedianIndices)):
        Medians.append(Joint[MedianIndices[j]])
    medianTrajectories = np.array(Medians)   
    
    return medianTrajectories

In [ ]:
def getLatent(medianTrajectories):
    meanJointX = []
    for i in range(len(medianTrajectories[0])):
        meanJointX.append(np.mean(medianTrajectories[:,i,0]*-1)) #due to x-axis inversion

    meanJointY = []
    for i in range(len(medianTrajectories[0])):
        meanJointY.append(np.mean(medianTrajectories[:,i,1]))

    meanJointZ = []
    for i in range(len(medianTrajectories[0])):
        meanJointZ.append(np.mean(medianTrajectories[:,i,2]))

    latent = np.array((meanJointX,meanJointY,meanJointZ)).T

    tRange = medianTrajectories[:,-1,-1]-medianTrajectories[:,0,-1]

    latentTime = np.linspace(0, np.mean(tRange), len(medianTrajectories[0,:,-1]))

    latentStd = np.ones_like(latent)*50

    return latent, latentTime, latentStd

In [ ]:
def doDTW(Joint, WarpFolder):
    
    Warps = []
    for i in range (0,len(Joint)):
        observed = np.array(Joint[i].iloc[:,0:3])
        if observed[:,0].all() < 0:
            observed[:,0] *= -1 #due to  x-axis inversion

        observedTime = np.array(Joint[i].loc[:,'Time'])
        observedTimeRange = observedTime[-1]-observedTime[0]
        observedTimeStep = (observedTime[1]-observedTime[0])
        observedTime = np.linspace(0, observedTimeRange, len(Joint[i].loc[:,'Time']))

        if len(observed)<=len(latent):
            
            odi, op = dynTimeWarp.dynTimeWarp(observed,latent,latentStd,transitions=[(1,1,0),(0,1,2)], 
            firstTransition=[(1,1,3),(0,1,0)], lastTransition=[(1,1,0),(0,1,np.log(2.0))])

            
            
            plt.clf()
            for ci in range(3):
                plt.subplot(3,1,ci+1)
                #plt.hold(True)
                plt.rcParams['figure.figsize'] = [8, 4]
                plt.plot(latentTime,latent[:,ci],linewidth=3,label="target")
                plt.plot(observedTime,observed[:,ci],linewidth=2.0,marker="o",linestyle="--",label="signal (y)")

                
                wy=np.array([observed[i[1]] for i in op])
                #print(wy)
                wt=[latentTime[i[0]] for i in op]
                #print(wt)
                plt.plot(wt,wy[:,ci],color="red",marker="o",label="opt. warp")
                plt.legend()

                plt.savefig(WarpFolder + 'Warp' + str(i) + ".png")

        elif len(observed)>len(latent):
            odi, op = dynTimeWarp.dynTimeWarp(observed,latent,latentStd,transitions=[(1,1,0),(1,0,0)])
            
        
            plt.clf()
            for ci in range(3):
                plt.subplot(3,1,ci+1)
                #plt.hold(True)
                plt.rcParams['figure.figsize'] = [8, 4]
                plt.plot(latentTime,latent[:,ci],linewidth=3,label="target")
                plt.plot(observedTime,observed[:,ci],linewidth=2.0,marker="o",linestyle="--",label="signal (y)")

                
                wy=np.array([observed[i[1]] for i in op])
                #print(wy)
                wt=[latentTime[i[0]] for i in op]
                #print(wt)
                plt.plot(wt,wy[:,ci],color="red",marker="o",label="opt. warp")
                plt.legend()

                plt.savefig(WarpFolder + 'Warp' + str(i) + ".png")
                #plt.savefig(WarpFolder + 'VelWarp' + str(i) + ".png")


        Warps.append(np.column_stack((wy,wt)))
    
    return Warps

In [ ]:
def interpolateWarp(Warps, InterpolResFolder):

    WarpsXNewList = []
    WarpsYNewList = []
    WarpsZNewList = []
    
    for i in range(len(Warps)):

        time = np.array((Warps[i][:,3]))

        WarpsX = np.array((Warps[i][:,0]))
        WarpsY = np.array((Warps[i][:,2]))
        WarpsZ = np.array((Warps[i][:,1]))
        
        fX = interpolate.interp1d(time,WarpsX)
        fY = interpolate.interp1d(time,WarpsY)
        fZ = interpolate.interp1d(time,WarpsZ)
        
        newTime = np.linspace(np.min(time), np.max(time), 50)

        WarpsXNew = fX(newTime)
        WarpsYNew = fY(newTime)
        WarpsZNew = fZ(newTime)

        WarpsXNewList.append(WarpsXNew)
        WarpsYNewList.append(WarpsYNew)
        WarpsZNewList.append(WarpsZNew)
        

        plt.clf()
        fig, axs = plt.subplots(3)
        axs[0].plot(time,WarpsX,'o', newTime, WarpsXNew, '*r')
        axs[1].plot(time,WarpsY,'o', newTime, WarpsYNew, '*r')
        axs[2].plot(time,WarpsZ,'o', newTime, WarpsZNew, '*r')
    
        plt.savefig(InterpolResFolder + 'InterpolatedTrajectory' + str(i) + ".png")

        
    return WarpsXNewList, WarpsYNewList, WarpsZNewList, newTime

In [ ]:
def plotMeanVar(InterpolatedWarps, MeanTrajFolder):
    interpolWarps = np.array((InterpolatedWarps[0],InterpolatedWarps[1],InterpolatedWarps[2])).T
    print(interpolWarps.shape)


    interpolWarpMeanX = []
    for i in range(len(InterpolatedWarps[3])):
        interpolWarpMeanX.append(np.mean(interpolWarps[i,:,0]))

    interpolWarpMeanY = []
    for i in range(len(InterpolatedWarps[3])):
        interpolWarpMeanY.append(np.mean(interpolWarps[i,:,2]))

    interpolWarpMeanZ = []
    for i in range(len(InterpolatedWarps[3])):
        interpolWarpMeanZ.append(np.mean(interpolWarps[i,:,1]))

    MeanTrajectories = np.array((interpolWarpMeanX,interpolWarpMeanY,interpolWarpMeanZ)).T


    interpolWarpStdX = []
    for i in range(len(InterpolatedWarps[3])):
        interpolWarpStdX.append(np.std(interpolWarps[i,:,0]))

    interpolWarpStdY = []
    for i in range(len(InterpolatedWarps[3])):
        interpolWarpStdY.append(np.std(interpolWarps[i,:,2]))

    interpolWarpStdZ = []
    for i in range(len(InterpolatedWarps[3])):
        interpolWarpStdZ.append(np.std(interpolWarps[i,:,1]))

    TrajectoryVariances = np.array((interpolWarpStdX,interpolWarpStdY,interpolWarpStdZ)).T



    plt.clf()
    fig, axs = plt.subplots(3, figsize=(8,10))
    #axs[0].set_ylim(bottom = -95, top = -60)
    axs[0].plot(InterpolatedWarps[3],MeanTrajectories[:,0])
    #axs[1].set_ylim(bottom = 5, top = 40)
    axs[2].plot(InterpolatedWarps[3],MeanTrajectories[:,1])
    #axs[2].set_ylim(bottom = 95, top = 110)
    axs[1].plot(InterpolatedWarps[3],MeanTrajectories[:,2])

    axs[0].plot(InterpolatedWarps[3],MeanTrajectories[:,0] - TrajectoryVariances[:,0])
    axs[2].plot(InterpolatedWarps[3],MeanTrajectories[:,1] - TrajectoryVariances[:,1])
    axs[1].plot(InterpolatedWarps[3],MeanTrajectories[:,2] - TrajectoryVariances[:,2])

    axs[0].plot(InterpolatedWarps[3],MeanTrajectories[:,0] + TrajectoryVariances[:,0])
    axs[2].plot(InterpolatedWarps[3],MeanTrajectories[:,1] + TrajectoryVariances[:,1])
    axs[1].plot(InterpolatedWarps[3],MeanTrajectories[:,2] + TrajectoryVariances[:,2])

    plt.savefig(MeanTrajFolder + 'MeanVarTrajectory.png')

    np.savetxt(MeanTrajFolder + 'Mean.csv', MeanTrajectories, delimiter = ',')
    np.savetxt(MeanTrajFolder + 'Variance.csv', TrajectoryVariances, delimiter = ',')
    
    #plt.savefig(MeanTrajFolder + 'MeanVarVel.png')
    #np.savetxt(MeanTrajFolder + 'MeanVel.csv', MeanTrajectories, delimiter = ',')
    #np.savetxt(MeanTrajFolder + 'VarianceVel.csv', TrajectoryVariances, delimiter = ',')

    return MeanTrajectories

In [ ]:
def plotMean3D(MeanTrajFolder):
    plt.clf()
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111, projection='3d')

    for i in range(len(InterpolatedWarps[0])):

        ax.plot(InterpolatedWarps[0][i], InterpolatedWarps[1][i], InterpolatedWarps[2][i], '0.7')

    ax.plot(InterpolatedWarps[0][0], InterpolatedWarps[1][0], InterpolatedWarps[2][0], '0.7', label = 'Warped Trajectories')

    ax.plot(MeanTrajectories[:,0], MeanTrajectories[:,2], MeanTrajectories[:,1], linewidth = 4, label = 'Mean Trajectory')
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.legend()

    plt.savefig(MeanTrajFolder + 'MeanVarTrajectory3D.png')


In [ ]:
def unpickleControllerDFs(infile = '.../'):

    infile = open(infile,'rb')
    controllerDFs = pickle.load(infile)
    infile.close()

    return controllerDFs

In [ ]:
def calcPlotVelocity(meanTrajectory, T):
    t = np.linspace(0,T)
    dt=t[1]
    diff = meanTrajectory.diff()

    coords = [c for c in meanTrajectory.columns ]
    vel = np.linalg.norm(diff[coords], axis=1)/dt
    vel = np.array(vel)
    print(vel)

    plt.plot(t, vel)

    return vel

In [ ]:
def calcPlotAcceleration(vel, T):
    vel = pd.DataFrame(vel)
    t = np.linspace(0,T)
    dt=t[1]
    diff = vel.diff()

    coords = [c for c in vel.columns ]
    acc = np.linalg.norm(diff[coords], axis=1)/dt
    acc = np.array(acc)

    plt.plot(t, acc)

    return acc

In [ ]:
#use this to apply filtering and derivation at once
def getVelAcc(meanTrajectory, meanTrajFolder, T):

    VelFilt = savgol_filter(meanTrajectory, 11, 2, 1, axis=0)
    plt.clf()
    for ci in range(3):
        plt.subplot(3,1,ci+1)
        plt.rcParams['figure.figsize'] = [8, 4]
        plt.plot(T, VelFilt[:,ci],linewidth=3,label="Velocity")

    plt.savefig(meanTrajFolder + 'MeanVel.png')
    np.savetxt(meanTrajFolder + 'MeanVel.csv', VelFilt, delimiter = ',')

    AccFilt = savgol_filter(meanTrajectory, 17, 2, 2, axis=0)
    plt.clf()
    for ci in range(3):
        plt.subplot(3,1,ci+1)

        plt.plot(T, AccFilt[:,ci],linewidth=3,label="Acceleration")
        
    plt.savefig(meanTrajFolder + 'MeanAcc.png')
    np.savetxt(meanTrajFolder + 'MeanAcc.csv', AccFilt, delimiter = ',')

    return VelFilt, AccFilt

In [ ]:
def insertTimes(Vels, Times):
    for i in range(len(Vels)):
        Vels[i].insert(3, 'Time', Times[i])

    return Vels

In [ ]:
def makeVels(condControllerDFs):

    condVels = []
    for i in range(len(condControllerDFs)):
        condControllerArray = np.asarray(condControllerDFs[i])
        condVels.append(finiteDifferenceHigherOrder(condControllerArray[:,0], condControllerArray[:,1], condControllerArray[:,2]))
        #condVels[i] = pd.DataFrame(condVels[i])

    return condVels

In [ ]:
def finiteDifferenceHigherOrder(xVel, yVel, zVel):
    
    """ 
        Caclulate xDerivative by 2nd/4th order finite difference method.
        http://www.math.ubc.ca/~jfeng/CHBE553/Example7/Formulae.pdf 
    """

    step = 0.02
    xDerivative = np.empty_like(xVel) 

    xDerivative[:2] = ((-xVel[2:4] + 4*xVel[1:3] - 3*xVel[:2]) / (2*step)) # forward
    xDerivative[-2:] = ((3*xVel[-2:] - 4*xVel[-3:-1] + xVel[-4:-2]) / (2*step)) # backward
    xDerivative[2:-2] = ((-xVel[4:] + 8*(xVel[3:-1] - xVel[1:-3]) + xVel[:-4]) /(12*step)) # central

    yDerivative = np.empty_like(yVel) 

    yDerivative[:2] = ((-yVel[2:4] + 4*yVel[1:3] - 3*yVel[:2]) / (2*step)) 
    yDerivative[-2:] = ((3*yVel[-2:] - 4*yVel[-3:-1] + yVel[-4:-2]) / (2*step)) 
    yDerivative[2:-2] = ((-yVel[4:] + 8*(yVel[3:-1] - yVel[1:-3]) + yVel[:-4]) /(12*step)) 


    zDerivative = np.empty_like(zVel) 

    zDerivative[:2] = ((-zVel[2:4] + 4*zVel[1:3] - 3*zVel[:2]) / (2*step)) 
    zDerivative[-2:] = ((3*zVel[-2:] - 4*zVel[-3:-1] + zVel[-4:-2]) / (2*step)) 
    zDerivative[2:-2] = ((-zVel[4:] + 8*(zVel[3:-1] - zVel[1:-3]) + zVel[:-4]) /(12*step)) 

    allDerivatives = np.column_stack((xDerivative,yDerivative,zDerivative))

    return allDerivatives


In [ ]:
#nach oben zu den Funktionen setzen
def DoAnalysis(cond, folder):
    Medians = getMedianDuration(cond)

    latent, latentTime, latentStd = getLatent(Medians)
    Warps = doDTW(cond, WarpFolder = path + 'WarpResults/'+folder+'/Controller/')

    InterpolatedWarps = interpolateWarp(Warps, InterpolResFolder = path + 'InterpolResults/'+folder+'/Controller/')

    MeanTrajectories = plotMeanVar(InterpolatedWarps, MeanTrajFolder = path + 'MeanTrajectories/'+folder+'/Controller/')

    Mean3D = plotMean3D(MeanTrajFolder = path + 'MeanTrajectories/'+folder+'/Controller/')


Get the arm moves

In [ ]:
#During the experiment, 17 csv files are generated for one participant, one for each block -> read all of these csv's and create dfs

pathVPSData = '.../'
B3 = pd.read_csv(pathVPSData+'P._3_log.csv', sep = ';')
B4 = pd.read_csv(pathVPSData+'P._4_log.csv', sep = ';')
B5 = pd.read_csv(pathVPSData+'P._5_log.csv', sep = ';')
B6 = pd.read_csv(pathVPSData+'P._6_log.csv', sep = ';')
B7 = pd.read_csv(pathVPSData+'P._7_log.csv', sep = ';')
B8 = pd.read_csv(pathVPSData+'P._8_log.csv', sep = ';')
B9 = pd.read_csv(pathVPSData+'P._9_log.csv', sep = ';')
B10 = pd.read_csv(pathVPSData+'P._10_log.csv', sep = ';')
B11 = pd.read_csv(pathVPSData+'P._11_log.csv', sep = ';')
B12 = pd.read_csv(pathVPSData+'P._12_log.csv', sep = ';')
B3 = pd.read_csv(pathVPSData+'P._3_log.csv', sep = ';')
B14 = pd.read_csv(pathVPSData+'P._14_log.csv', sep = ';')
B15 = pd.read_csv(pathVPSData+'P._15_log.csv', sep = ';')
B16 = pd.read_csv(pathVPSData+'P._16_log.csv', sep = ';')
B17 = pd.read_csv(pathVPSData+'P._17_log.csv', sep = ';')


AllBlocks = (B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B3,B14,B15,B16,B17)

In [ ]:
path = '.../'

In [ ]:
makeArmMoveFolders('P.')

In [ ]:
makeResultFolders('P./')

In [ ]:
makeAllFolders()
makeControllerFolders()

In [ ]:
def main(participant):
    SaveMovements(AllBlocks, pathArmMoves = '.../'+participant+'.../')

if __name__=="__main__":
    main('P.')

get perturbation onsets

In [ ]:
""" cond2LeftPertOnsets = flattenList(cond2LeftPertOnsets)
cond2RightPertOnsets = flattenList(cond2RightPertOnsets)
cond3LeftPertOnsets = flattenList(cond3LeftPertOnsets)
cond3RightPertOnsets = flattenList(cond3RightPertOnsets)
cond4LeftPertOnsets = flattenList(cond4LeftPertOnsets)
cond4RightPertOnsets = flattenList(cond4RightPertOnsets)
cond5LeftPertOnsets = flattenList(cond5LeftPertOnsets)
cond5RightPertOnsets = flattenList(cond5RightPertOnsets)
cond6LeftPertOnsets = flattenList(cond6LeftPertOnsets)
cond6RightPertOnsets = flattenList(cond6RightPertOnsets)
cond7LeftPertOnsets = flattenList(cond7LeftPertOnsets)
cond7RightPertOnsets = flattenList(cond7RightPertOnsets)  """

In [ ]:
""" meanPertOnsets = {'P125L':np.mean(cond2LeftPertOnsets), 'P125R':np.mean(cond2RightPertOnsets),
'P150L':np.mean(cond3LeftPertOnsets), 'P150R':np.mean(cond3RightPertOnsets), 
'P175L':np.mean(cond4LeftPertOnsets), 'P175R':np.mean(cond4RightPertOnsets), 
'P200L':np.mean(cond5LeftPertOnsets), 'P200R':np.mean(cond5RightPertOnsets),
'P225L':np.mean(cond6LeftPertOnsets), 'P225R':np.mean(cond6RightPertOnsets), 
'P250L':np.mean(cond7LeftPertOnsets), 'P250R':np.mean(cond7RightPertOnsets), } """

In [ ]:
""" stdPertOnsets = {'P125L':np.std(cond2LeftPertOnsets), 'P125R':np.std(cond2RightPertOnsets),
'P150L':np.std(cond3LeftPertOnsets), 'P150R':np.std(cond3RightPertOnsets), 
'P175L':np.std(cond4LeftPertOnsets), 'P175R':np.std(cond4RightPertOnsets), 
'P200L':np.std(cond5LeftPertOnsets), 'P200R':np.std(cond5RightPertOnsets),
'P225L':np.std(cond6LeftPertOnsets), 'P225R':np.std(cond6RightPertOnsets), 
'P250L':np.std(cond7LeftPertOnsets), 'P250R':np.std(cond7RightPertOnsets), } """

In [ ]:
""" with open('/Users/ruslan/Dropbox/Mein Mac (Air-von-Ruslan)/Desktop/Uni (Neu)/Masterarbeit/DataAnalysis/Results/P13/PertCorrectionOnsets/meanPertOnsets.csv', 'w') as f:
    for key in meanPertOnsets.keys():
        f.write("%s,%s\n"%(key,meanPertOnsets[key])) """

In [ ]:
""" with open('/Users/ruslan/Dropbox/Mein Mac (Air-von-Ruslan)/Desktop/Uni (Neu)/Masterarbeit/DataAnalysis/Results/P6/PertCorrectionOnsets/stdPertOnsets.csv', 'w') as f:
    for key in stdPertOnsets.keys():
        f.write("%s,%s\n"%(key,stdPertOnsets[key])) """

get ball and controller moves

In [ ]:
(cond1BallMoves, cond2BallMovesLeft, cond2BallMovesRight, cond3BallMovesLeft, cond3BallMovesRight, cond4BallMovesLeft, cond4BallMovesRight, cond5BallMovesLeft, 
cond5BallMovesRight, cond6BallMovesLeft, cond6BallMovesRight, cond7BallMovesLeft, cond7BallMovesRight) = getViveMovements(AllBlocks, ViveX = 'Ball_x', ViveZ = 'Ball_y')

In [ ]:
cond1BallMoves = flattenList(cond1BallMoves)
cond2BallMovesLeft = flattenList(cond2BallMovesLeft)
cond2BallMovesRight = flattenList(cond2BallMovesRight)
cond3BallMovesLeft = flattenList(cond3BallMovesLeft)
cond3BallMovesRight = flattenList(cond3BallMovesRight)
cond4BallMovesLeft = flattenList(cond4BallMovesLeft)
cond4BallMovesRight = flattenList(cond4BallMovesRight)
cond5BallMovesLeft = flattenList(cond5BallMovesLeft)
cond5BallMovesRight = flattenList(cond5BallMovesRight)
cond6BallMovesLeft = flattenList(cond6BallMovesLeft)
cond6BallMovesRight = flattenList(cond6BallMovesRight)
cond7BallMovesLeft = flattenList(cond7BallMovesLeft)
cond7BallMovesRight = flattenList(cond7BallMovesRight)


In [ ]:
cond1BallMoves = MakeViveMoveArray(cond1BallMoves)
cond2BallMovesLeft = MakeViveMoveArray(cond2BallMovesLeft)
cond2BallMovesRight = MakeViveMoveArray(cond2BallMovesRight)
cond3BallMovesLeft = MakeViveMoveArray(cond3BallMovesLeft)
cond3BallMovesRight = MakeViveMoveArray(cond3BallMovesRight)
cond4BallMovesLeft = MakeViveMoveArray(cond4BallMovesLeft)
cond4BallMovesRight = MakeViveMoveArray(cond4BallMovesRight)
cond5BallMovesLeft = MakeViveMoveArray(cond5BallMovesLeft)
cond5BallMovesRight = MakeViveMoveArray(cond5BallMovesRight)
cond6BallMovesLeft = MakeViveMoveArray(cond6BallMovesLeft)
cond6BallMovesRight = MakeViveMoveArray(cond6BallMovesRight)
cond7BallMovesLeft = MakeViveMoveArray(cond7BallMovesLeft)
cond7BallMovesRight = MakeViveMoveArray(cond7BallMovesRight)

In [ ]:
T = pd.read_csv(pathVPSData + 'P._1_matrixT.csv', sep = ';', header=None)

In [ ]:
TMt = getTMatrix(T)

In [ ]:
cond1BallMoves = transformViveCoordinates(cond1BallMoves)
cond2BallMovesLeft = transformViveCoordinates(cond2BallMovesLeft)
cond2BallMovesRight = transformViveCoordinates(cond2BallMovesRight)
cond3BallMovesLeft = transformViveCoordinates(cond3BallMovesLeft)
cond3BallMovesRight = transformViveCoordinates(cond3BallMovesRight)
cond4BallMovesLeft = transformViveCoordinates(cond4BallMovesLeft)
cond4BallMovesRight = transformViveCoordinates(cond4BallMovesRight)
cond5BallMovesLeft = transformViveCoordinates(cond5BallMovesLeft)
cond5BallMovesRight = transformViveCoordinates(cond5BallMovesRight)
cond6BallMovesLeft = transformViveCoordinates(cond6BallMovesLeft)
cond6BallMovesRight = transformViveCoordinates(cond6BallMovesRight)
cond7BallMovesLeft = transformViveCoordinates(cond7BallMovesLeft)
cond7BallMovesRight = transformViveCoordinates(cond7BallMovesRight)

In [ ]:
cond2LeftBallDistances = getBallDistances(cond2BallMovesLeft)
cond2RightBallDistances = getBallDistances(cond2BallMovesRight)
cond3LeftBallDistances = getBallDistances(cond3BallMovesLeft)
cond3RightBallDistances = getBallDistances(cond3BallMovesRight)
cond4LeftBallDistances = getBallDistances(cond4BallMovesLeft)
cond4RightBallDistances = getBallDistances(cond4BallMovesRight)
cond5LeftBallDistances = getBallDistances(cond5BallMovesLeft)
cond5RightBallDistances = getBallDistances(cond5BallMovesRight)
cond6LeftBallDistances = getBallDistances(cond6BallMovesLeft)
cond6RightBallDistances = getBallDistances(cond6BallMovesRight)
cond7LeftBallDistances = getBallDistances(cond7BallMovesLeft)
cond7RightBallDistances = getBallDistances(cond7BallMovesRight)

In [ ]:
meanBallDistances = {'P125L':np.mean(cond2LeftBallDistances), 'P125R':np.mean(cond2RightBallDistances),
'P150L':np.mean(cond3LeftBallDistances), 'P150R':np.mean(cond3RightBallDistances), 
'P175L':np.mean(cond4LeftBallDistances), 'P175R':np.mean(cond4RightBallDistances), 
'P200L':np.mean(cond5LeftBallDistances), 'P200R':np.mean(cond5RightBallDistances),
'P225L':np.mean(cond6LeftBallDistances), 'P225R':np.mean(cond6RightBallDistances), 
'P250L':np.mean(cond7LeftBallDistances), 'P250R':np.mean(cond7RightBallDistances), }

In [ ]:
stdBallDistances = {'P125L':np.std(cond2LeftBallDistances), 'P125R':np.std(cond2RightBallDistances),
'P150L':np.std(cond3LeftBallDistances), 'P150R':np.std(cond3RightBallDistances), 
'P175L':np.std(cond4LeftBallDistances), 'P175R':np.std(cond4RightBallDistances), 
'P200L':np.std(cond5LeftBallDistances), 'P200R':np.std(cond5RightBallDistances),
'P225L':np.std(cond6LeftBallDistances), 'P225R':np.std(cond6RightBallDistances), 
'P250L':np.std(cond7LeftBallDistances), 'P250R':np.std(cond7RightBallDistances), }

In [ ]:
with open('.../meanBallDistances.csv', 'w') as f:
    for key in meanBallDistances.keys():
        f.write("%s,%s\n"%(key,meanBallDistances[key]))

In [ ]:
with open('.../stdBallDistances.csv', 'w') as f:
    for key in stdBallDistances.keys():
        f.write("%s,%s\n"%(key,stdBallDistances[key]))

In [ ]:
meansPath = '.../'

In [ ]:
(cond1ControllerMoves, cond2ControllerMovesLeft, cond2ControllerMovesRight, cond3ControllerMovesLeft, cond3ControllerMovesRight, cond4ControllerMovesLeft, cond4ControllerMovesRight, cond5ControllerMovesLeft, 
cond5ControllerMovesRight, cond6ControllerMovesLeft, cond6ControllerMovesRight, cond7ControllerMovesLeft, cond7ControllerMovesRight) = getViveMovements(AllBlocks, ViveX = 'Controller_x', ViveZ = 'Controller_y')

In [ ]:
cond1ControllerMoves = flattenList(cond1ControllerMoves)

cond2ControllerMovesLeft = flattenList(cond2ControllerMovesLeft)
cond2ControllerMovesRight = flattenList(cond2ControllerMovesRight)

cond3ControllerMovesLeft = flattenList(cond3ControllerMovesLeft)
cond3ControllerMovesRight = flattenList(cond3ControllerMovesRight)

cond4ControllerMovesLeft = flattenList(cond4ControllerMovesLeft)
cond4ControllerMovesRight = flattenList(cond4ControllerMovesRight)

cond5ControllerMovesLeft = flattenList(cond5ControllerMovesLeft)
cond5ControllerMovesRight = flattenList(cond5ControllerMovesRight)

cond6ControllerMovesLeft = flattenList(cond6ControllerMovesLeft)
cond6ControllerMovesRight = flattenList(cond6ControllerMovesRight)

cond7ControllerMovesLeft = flattenList(cond7ControllerMovesLeft)
cond7ControllerMovesRight = flattenList(cond7ControllerMovesRight)

In [ ]:
cond1ControllerMoves = MakeViveMoveArray(cond1ControllerMoves)

cond2ControllerMovesLeft = MakeViveMoveArray(cond2ControllerMovesLeft)
cond2ControllerMovesRight = MakeViveMoveArray(cond2ControllerMovesRight)

cond3ControllerMovesLeft = MakeViveMoveArray(cond3ControllerMovesLeft)
cond3ControllerMovesRight = MakeViveMoveArray(cond3ControllerMovesRight)

cond4ControllerMovesLeft = MakeViveMoveArray(cond4ControllerMovesLeft)
cond4ControllerMovesRight = MakeViveMoveArray(cond4ControllerMovesRight)

cond5ControllerMovesLeft = MakeViveMoveArray(cond5ControllerMovesLeft)
cond5ControllerMovesRight = MakeViveMoveArray(cond5ControllerMovesRight)

cond6ControllerMovesLeft = MakeViveMoveArray(cond6ControllerMovesLeft)
cond6ControllerMovesRight = MakeViveMoveArray(cond6ControllerMovesRight)

cond7ControllerMovesLeft = MakeViveMoveArray(cond7ControllerMovesLeft)
cond7ControllerMovesRight = MakeViveMoveArray(cond7ControllerMovesRight)

In [ ]:
cond1ControllerTimes = collectTimes(cond1ControllerMoves)

cond2LeftControllerTimes = collectTimes(cond2ControllerMovesLeft)
cond2RightControllerTimes = collectTimes(cond2ControllerMovesRight)

cond3LeftControllerTimes = collectTimes(cond3ControllerMovesLeft)
cond3RightControllerTimes = collectTimes(cond3ControllerMovesRight)

cond4LeftControllerTimes = collectTimes(cond4ControllerMovesLeft)
cond4RightControllerTimes = collectTimes(cond4ControllerMovesRight)

cond5LeftControllerTimes = collectTimes(cond5ControllerMovesLeft)
cond5RightControllerTimes = collectTimes(cond5ControllerMovesRight)

cond6LeftControllerTimes = collectTimes(cond6ControllerMovesLeft)
cond6RightControllerTimes = collectTimes(cond6ControllerMovesRight)

cond7LeftControllerTimes = collectTimes(cond7ControllerMovesLeft)
cond7RightControllerTimes = collectTimes(cond7ControllerMovesRight)


In [ ]:
cond1ControllerMoves = transformViveCoordinates(cond1ControllerMoves)

cond2ControllerMovesLeft = transformViveCoordinates(cond2ControllerMovesLeft)
cond2ControllerMovesRight = transformViveCoordinates(cond2ControllerMovesRight)

cond3ControllerMovesLeft = transformViveCoordinates(cond3ControllerMovesLeft)
cond3ControllerMovesRight = transformViveCoordinates(cond3ControllerMovesRight)

cond4ControllerMovesLeft = transformViveCoordinates(cond4ControllerMovesLeft)
cond4ControllerMovesRight = transformViveCoordinates(cond4ControllerMovesRight)

cond5ControllerMovesLeft = transformViveCoordinates(cond5ControllerMovesLeft)
cond5ControllerMovesRight = transformViveCoordinates(cond5ControllerMovesRight)

cond6ControllerMovesLeft = transformViveCoordinates(cond6ControllerMovesLeft)
cond6ControllerMovesRight = transformViveCoordinates(cond6ControllerMovesRight)

cond7ControllerMovesLeft = transformViveCoordinates(cond7ControllerMovesLeft)
cond7ControllerMovesRight = transformViveCoordinates(cond7ControllerMovesRight)

In [ ]:
cond1ControllerDFs = makeControllerDF(cond1ControllerMoves, cond1ControllerTimes)

cond2LeftControllerDFs = makeControllerDF(cond2ControllerMovesLeft, cond2LeftControllerTimes)
cond2RightControllerDFs = makeControllerDF(cond2ControllerMovesRight, cond2RightControllerTimes)

cond3LeftControllerDFs = makeControllerDF(cond3ControllerMovesLeft, cond3LeftControllerTimes)
cond3RightControllerDFs = makeControllerDF(cond3ControllerMovesRight, cond3RightControllerTimes)

cond4LeftControllerDFs = makeControllerDF(cond4ControllerMovesLeft, cond4LeftControllerTimes)
cond4RightControllerDFs = makeControllerDF(cond4ControllerMovesRight, cond4RightControllerTimes)

cond5LeftControllerDFs = makeControllerDF(cond5ControllerMovesLeft, cond5LeftControllerTimes)
cond5RightControllerDFs = makeControllerDF(cond5ControllerMovesRight, cond5RightControllerTimes)

cond6LeftControllerDFs = makeControllerDF(cond6ControllerMovesLeft, cond6LeftControllerTimes)
cond6RightControllerDFs = makeControllerDF(cond6ControllerMovesRight, cond6RightControllerTimes)

cond7LeftControllerDFs = makeControllerDF(cond7ControllerMovesLeft, cond7LeftControllerTimes)
cond7RightControllerDFs = makeControllerDF(cond7ControllerMovesRight, cond7RightControllerTimes)


In [ ]:
pathVPSData = '.../'
#B1 = pd.read_csv(pathVPSData+'P._1_log.csv', sep = ';')
#B2 = pd.read_csv(pathVPSData+'P._2_log.csv', sep = ';')
B3 = pd.read_csv(pathVPSData+'P._3_log.csv', sep = ';')
B4 = pd.read_csv(pathVPSData+'P._4_log.csv', sep = ';')
B5 = pd.read_csv(pathVPSData+'P._5_log.csv', sep = ';')
B6 = pd.read_csv(pathVPSData+'P._6_log.csv', sep = ';')
B7 = pd.read_csv(pathVPSData+'P._7_log.csv', sep = ';')
B8 = pd.read_csv(pathVPSData+'P._8_log.csv', sep = ';')
B9 = pd.read_csv(pathVPSData+'P._9_log.csv', sep = ';')
B10 = pd.read_csv(pathVPSData+'P._10_log.csv', sep = ';')
B11 = pd.read_csv(pathVPSData+'P._11_log.csv', sep = ';')
B12 = pd.read_csv(pathVPSData+'P._12_log.csv', sep = ';')
B3 = pd.read_csv(pathVPSData+'P._3_log.csv', sep = ';')
B14 = pd.read_csv(pathVPSData+'P._14_log.csv', sep = ';')
B15 = pd.read_csv(pathVPSData+'P._15_log.csv', sep = ';')
B16 = pd.read_csv(pathVPSData+'P._16_log.csv', sep = ';')
B17 = pd.read_csv(pathVPSData+'P._17_log.csv', sep = ';')


AllBlocks = (B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B3,B14,B15,B16,B17)

path = '.../'

T = pd.read_csv(pathVPSData + 'P._1_matrixT.csv', sep = ';', header=None)

TMt = getTMatrix(T)

meansPath = '.../'

(cond1ControllerMoves, cond2ControllerMovesLeft, cond2ControllerMovesRight, cond3ControllerMovesLeft, cond3ControllerMovesRight, cond4ControllerMovesLeft, cond4ControllerMovesRight, cond5ControllerMovesLeft, 
cond5ControllerMovesRight, cond6ControllerMovesLeft, cond6ControllerMovesRight, cond7ControllerMovesLeft, cond7ControllerMovesRight) = getViveMovements(AllBlocks, ViveX = 'Controller_x', ViveZ = 'Controller_y')

cond1ControllerMoves = flattenList(cond1ControllerMoves)

cond2ControllerMovesLeft = flattenList(cond2ControllerMovesLeft)
cond2ControllerMovesRight = flattenList(cond2ControllerMovesRight)

cond3ControllerMovesLeft = flattenList(cond3ControllerMovesLeft)
cond3ControllerMovesRight = flattenList(cond3ControllerMovesRight)

cond4ControllerMovesLeft = flattenList(cond4ControllerMovesLeft)
cond4ControllerMovesRight = flattenList(cond4ControllerMovesRight)

cond5ControllerMovesLeft = flattenList(cond5ControllerMovesLeft)
cond5ControllerMovesRight = flattenList(cond5ControllerMovesRight)

cond6ControllerMovesLeft = flattenList(cond6ControllerMovesLeft)
cond6ControllerMovesRight = flattenList(cond6ControllerMovesRight)

cond7ControllerMovesLeft = flattenList(cond7ControllerMovesLeft)
cond7ControllerMovesRight = flattenList(cond7ControllerMovesRight)

cond1ControllerMoves = MakeViveMoveArray(cond1ControllerMoves)

cond2ControllerMovesLeft = MakeViveMoveArray(cond2ControllerMovesLeft)
cond2ControllerMovesRight = MakeViveMoveArray(cond2ControllerMovesRight)

cond3ControllerMovesLeft = MakeViveMoveArray(cond3ControllerMovesLeft)
cond3ControllerMovesRight = MakeViveMoveArray(cond3ControllerMovesRight)

cond4ControllerMovesLeft = MakeViveMoveArray(cond4ControllerMovesLeft)
cond4ControllerMovesRight = MakeViveMoveArray(cond4ControllerMovesRight)

cond5ControllerMovesLeft = MakeViveMoveArray(cond5ControllerMovesLeft)
cond5ControllerMovesRight = MakeViveMoveArray(cond5ControllerMovesRight)

cond6ControllerMovesLeft = MakeViveMoveArray(cond6ControllerMovesLeft)
cond6ControllerMovesRight = MakeViveMoveArray(cond6ControllerMovesRight)

cond7ControllerMovesLeft = MakeViveMoveArray(cond7ControllerMovesLeft)
cond7ControllerMovesRight = MakeViveMoveArray(cond7ControllerMovesRight)



cond1ControllerTimes = collectTimes(cond1ControllerMoves)

cond2LeftControllerTimes = collectTimes(cond2ControllerMovesLeft)
cond2RightControllerTimes = collectTimes(cond2ControllerMovesRight)

cond3LeftControllerTimes = collectTimes(cond3ControllerMovesLeft)
cond3RightControllerTimes = collectTimes(cond3ControllerMovesRight)

cond4LeftControllerTimes = collectTimes(cond4ControllerMovesLeft)
cond4RightControllerTimes = collectTimes(cond4ControllerMovesRight)

cond5LeftControllerTimes = collectTimes(cond5ControllerMovesLeft)
cond5RightControllerTimes = collectTimes(cond5ControllerMovesRight)

cond6LeftControllerTimes = collectTimes(cond6ControllerMovesLeft)
cond6RightControllerTimes = collectTimes(cond6ControllerMovesRight)

cond7LeftControllerTimes = collectTimes(cond7ControllerMovesLeft)
cond7RightControllerTimes = collectTimes(cond7ControllerMovesRight)



cond1ControllerMoves = transformViveCoordinates(cond1ControllerMoves)

cond2ControllerMovesLeft = transformViveCoordinates(cond2ControllerMovesLeft)
cond2ControllerMovesRight = transformViveCoordinates(cond2ControllerMovesRight)

cond3ControllerMovesLeft = transformViveCoordinates(cond3ControllerMovesLeft)
cond3ControllerMovesRight = transformViveCoordinates(cond3ControllerMovesRight)

cond4ControllerMovesLeft = transformViveCoordinates(cond4ControllerMovesLeft)
cond4ControllerMovesRight = transformViveCoordinates(cond4ControllerMovesRight)

cond5ControllerMovesLeft = transformViveCoordinates(cond5ControllerMovesLeft)
cond5ControllerMovesRight = transformViveCoordinates(cond5ControllerMovesRight)

cond6ControllerMovesLeft = transformViveCoordinates(cond6ControllerMovesLeft)
cond6ControllerMovesRight = transformViveCoordinates(cond6ControllerMovesRight)

cond7ControllerMovesLeft = transformViveCoordinates(cond7ControllerMovesLeft)
cond7ControllerMovesRight = transformViveCoordinates(cond7ControllerMovesRight)


cond1ControllerDFs = makeControllerDF(cond1ControllerMoves, cond1ControllerTimes)

cond2LeftControllerDFs = makeControllerDF(cond2ControllerMovesLeft, cond2LeftControllerTimes)
cond2RightControllerDFs = makeControllerDF(cond2ControllerMovesRight, cond2RightControllerTimes)

cond3LeftControllerDFs = makeControllerDF(cond3ControllerMovesLeft, cond3LeftControllerTimes)
cond3RightControllerDFs = makeControllerDF(cond3ControllerMovesRight, cond3RightControllerTimes)

cond4LeftControllerDFs = makeControllerDF(cond4ControllerMovesLeft, cond4LeftControllerTimes)
cond4RightControllerDFs = makeControllerDF(cond4ControllerMovesRight, cond4RightControllerTimes)

cond5LeftControllerDFs = makeControllerDF(cond5ControllerMovesLeft, cond5LeftControllerTimes)
cond5RightControllerDFs = makeControllerDF(cond5ControllerMovesRight, cond5RightControllerTimes)

cond6LeftControllerDFs = makeControllerDF(cond6ControllerMovesLeft, cond6LeftControllerTimes)
cond6RightControllerDFs = makeControllerDF(cond6ControllerMovesRight, cond6RightControllerTimes)

cond7LeftControllerDFs = makeControllerDF(cond7ControllerMovesLeft, cond7LeftControllerTimes)
cond7RightControllerDFs = makeControllerDF(cond7ControllerMovesRight, cond7RightControllerTimes)


cond1Vels = makeVels(cond1ControllerDFs)  

cond2LeftVels = makeVels(cond2LeftControllerDFs)
cond2RightVels = makeVels(cond2RightControllerDFs)

cond3LeftVels = makeVels(cond3LeftControllerDFs)
cond3RightVels = makeVels(cond3RightControllerDFs)

cond4LeftVels = makeVels(cond4LeftControllerDFs)
cond4RightVels = makeVels(cond4RightControllerDFs)

cond5LeftVels = makeVels(cond5LeftControllerDFs)
cond5RightVels = makeVels(cond5RightControllerDFs)

cond6LeftVels = makeVels(cond6LeftControllerDFs)
cond6RightVels = makeVels(cond6RightControllerDFs)

cond7LeftVels = makeVels(cond7LeftControllerDFs)
cond7RightVels = makeVels(cond7RightControllerDFs)



cond1Vels = insertTimes(cond1Vels, cond1ControllerTimes)

cond2LeftVels = insertTimes(cond2LeftVels, cond2LeftControllerTimes) 
cond2RightVels = insertTimes(cond2RightVels, cond2RightControllerTimes) 

cond3LeftVels = insertTimes(cond3LeftVels, cond3LeftControllerTimes)
cond3RightVels = insertTimes(cond3RightVels, cond3RightControllerTimes)

cond4LeftVels = insertTimes(cond4LeftVels, cond4LeftControllerTimes)
cond4RightVels = insertTimes(cond4RightVels, cond4RightControllerTimes)

cond5LeftVels = insertTimes(cond5LeftVels, cond5LeftControllerTimes)
cond5RightVels = insertTimes(cond5RightVels, cond5RightControllerTimes)

cond6LeftVels = insertTimes(cond6LeftVels, cond6LeftControllerTimes)
cond6RightVels = insertTimes(cond6RightVels, cond6RightControllerTimes)

cond7LeftVels = insertTimes(cond7LeftVels, cond7LeftControllerTimes)
cond7RightVels = insertTimes(cond7RightVels, cond7RightControllerTimes)

In [ ]:
DoAnalysis(cond = cond1ControllerDFs, folder = 'NoPerturbation')

DoAnalysis(cond = cond2LeftControllerDFs, folder = 'Perturbation125/Left')
DoAnalysis(cond = cond2RightControllerDFs, folder = 'Perturbation125/Right')

DoAnalysis(cond = cond3LeftControllerDFs, folder = 'Perturbation150/Left')
DoAnalysis(cond = cond3RightControllerDFs, folder = 'Perturbation150/Right')

DoAnalysis(cond = cond4LeftControllerDFs, folder = 'Perturbation175/Left')
DoAnalysis(cond = cond4RightControllerDFs, folder = 'Perturbation175/Right')

DoAnalysis(cond = cond5LeftControllerDFs, folder = 'Perturbation200/Left')
DoAnalysis(cond = cond5RightControllerDFs, folder = 'Perturbation200/Right')

DoAnalysis(cond = cond6LeftControllerDFs, folder = 'Perturbation225/Left')
DoAnalysis(cond = cond6RightControllerDFs, folder = 'Perturbation225/Right')

DoAnalysis(cond = cond7LeftControllerDFs, folder = 'Perturbation250/Left')
DoAnalysis(cond = cond7RightControllerDFs, folder = 'Perturbation250/Right')

get move durations

In [ ]:
cond1Durations = getMoveDurations(cond1ControllerMoves)

cond2LeftDurations = getMoveDurations(cond2ControllerMovesLeft)
cond2RightDurations = getMoveDurations(cond2ControllerMovesRight)

cond3LeftDurations = getMoveDurations(cond3ControllerMovesLeft)
cond3RightDurations = getMoveDurations(cond3ControllerMovesRight)

cond4LeftDurations = getMoveDurations(cond4ControllerMovesLeft)
cond4RightDurations = getMoveDurations(cond4ControllerMovesRight)

cond5LeftDurations = getMoveDurations(cond5ControllerMovesLeft)
cond5RightDurations = getMoveDurations(cond5ControllerMovesRight)

cond6LeftDurations = getMoveDurations(cond6ControllerMovesLeft)
cond6RightDurations = getMoveDurations(cond6ControllerMovesRight)

cond7LeftDurations = getMoveDurations(cond7ControllerMovesLeft)
cond7RightDurations = getMoveDurations(cond7ControllerMovesRight)



In [ ]:
meanMoveDurations = {'NoP':np.mean(cond1Durations), 'P125L':np.mean(cond2LeftDurations), 'P125R':np.mean(cond2RightDurations),
'P150L':np.mean(cond3LeftDurations), 'P150R':np.mean(cond3RightDurations), 
'P175L':np.mean(cond4LeftDurations), 'P175R':np.mean(cond4RightDurations), 
'P200L':np.mean(cond5LeftDurations), 'P200R':np.mean(cond5RightDurations),
'P225L':np.mean(cond6LeftDurations), 'P225R':np.mean(cond6RightDurations), 
'P250L':np.mean(cond7LeftDurations), 'P250R':np.mean(cond7RightDurations), }

In [ ]:
stdMoveDurations = {'NoP':np.std(cond1Durations),'P125L':np.std(cond2LeftDurations), 'P125R':np.std(cond2RightDurations),
'P150L':np.std(cond3LeftDurations), 'P150R':np.std(cond3RightDurations), 
'P175L':np.std(cond4LeftDurations), 'P175R':np.std(cond4RightDurations), 
'P200L':np.std(cond5LeftDurations), 'P200R':np.std(cond5RightDurations),
'P225L':np.std(cond6LeftDurations), 'P225R':np.std(cond6RightDurations), 
'P250L':np.std(cond7LeftDurations), 'P250R':np.std(cond7RightDurations), }

In [ ]:
with open('.../', 'w') as f:
    for key in meanMoveDurations.keys():
        f.write("%s,%s\n"%(key,meanMoveDurations[key]))

In [ ]:
with open('.../', 'w') as f:
    for key in stdMoveDurations.keys():
        f.write("%s,%s\n"%(key,stdMoveDurations[key]))

get success rates

In [ ]:
pathVPSData = '.../'
B1 = pd.read_csv(pathVPSData+'P._1_log.csv', sep = ';')
B2 = pd.read_csv(pathVPSData+'P._2_log.csv', sep = ';')
B3 = pd.read_csv(pathVPSData+'P._6_log.csv', sep = ';')
B4 = pd.read_csv(pathVPSData+'P._7_log.csv', sep = ';')
B5 = pd.read_csv(pathVPSData+'P._5_log.csv', sep = ';')
B6 = pd.read_csv(pathVPSData+'P._6_log.csv', sep = ';')
B7 = pd.read_csv(pathVPSData+'P._7_log.csv', sep = ';')
B8 = pd.read_csv(pathVPSData+'P._8_log.csv', sep = ';')
B9 = pd.read_csv(pathVPSData+'P._9_log.csv', sep = ';')
B10 = pd.read_csv(pathVPSData+'P._10_log.csv', sep = ';')
B11 = pd.read_csv(pathVPSData+'P._11_log.csv', sep = ';')
B12 = pd.read_csv(pathVPSData+'P._12_log.csv', sep = ';')
B13 = pd.read_csv(pathVPSData+'P._13_log.csv', sep = ';')
B14 = pd.read_csv(pathVPSData+'P._14_log.csv', sep = ';')
B15 = pd.read_csv(pathVPSData+'P._15_log.csv', sep = ';')
B16 = pd.read_csv(pathVPSData+'P._16_log.csv', sep = ';')
B17 = pd.read_csv(pathVPSData+'P._17_log.csv', sep = ';')

AllBlocksWithTraining = (B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15,B16,B17)

In [ ]:
(cond2TrialsLeft,cond2TrialsRight,cond3TrialsLeft,cond3TrialsRight,cond4TrialsLeft,cond4TrialsRight,cond5TrialsLeft,cond5TrialsRight,cond6TrialsLeft,cond6TrialsRight,cond7TrialsLeft,cond7TrialsRight, 
cond2LeftSuccesses, cond2RightSuccesses, cond3LeftSuccesses, cond3RightSuccesses, cond4LeftSuccesses, cond4RightSuccesses, cond5LeftSuccesses, cond5RightSuccesses, cond6LeftSuccesses, cond6RightSuccesses, cond7LeftSuccesses, cond7RightSuccesses) = getSuccessRates(AllBlocksWithTraining) 

In [ ]:
cond2LeftPropsPerBlock = successRatePerCondPerBlock(cond2LeftSuccesses, cond2TrialsLeft)
cond2RightPropsPerBlock = successRatePerCondPerBlock(cond2RightSuccesses, cond2TrialsRight)

cond3LeftPropsPerBlock = successRatePerCondPerBlock(cond3LeftSuccesses, cond3TrialsLeft)
cond3RightPropsPerBlock = successRatePerCondPerBlock(cond3RightSuccesses, cond3TrialsRight)

cond4LeftPropsPerBlock = successRatePerCondPerBlock(cond4LeftSuccesses, cond4TrialsLeft)
cond4RightPropsPerBlock = successRatePerCondPerBlock(cond4RightSuccesses, cond4TrialsRight)

cond5LeftPropsPerBlock = successRatePerCondPerBlock(cond5LeftSuccesses, cond5TrialsLeft)
cond5RightPropsPerBlock = successRatePerCondPerBlock(cond5RightSuccesses, cond5TrialsRight)

cond6LeftPropsPerBlock = successRatePerCondPerBlock(cond6LeftSuccesses, cond6TrialsLeft)
cond6RightPropsPerBlock = successRatePerCondPerBlock(cond6RightSuccesses, cond6TrialsRight)

cond7LeftPropsPerBlock = successRatePerCondPerBlock(cond7LeftSuccesses, cond7TrialsLeft)
cond7RightPropsPerBlock = successRatePerCondPerBlock(cond7RightSuccesses, cond7TrialsRight)

In [ ]:
nTrialsPerBlock, nSuccessesPerBlock,proportionsPerBlock = successRatePerBlock()

In [ ]:
# the success rates within each condition

cond2LeftSuccessProportion = successRatePerCondition(cond2LeftSuccesses, cond2TrialsLeft)
cond2RightSuccessProportion = successRatePerCondition(cond2RightSuccesses, cond2TrialsRight)

cond3LeftSuccessProportion = successRatePerCondition(cond3LeftSuccesses, cond3TrialsLeft)
cond3RightSuccessProportion = successRatePerCondition(cond3RightSuccesses, cond3TrialsRight)

cond4LeftSuccessProportion = successRatePerCondition(cond4LeftSuccesses, cond4TrialsLeft)
cond4RightSuccessProportion = successRatePerCondition(cond4RightSuccesses, cond4TrialsRight)

cond5LeftSuccessProportion = successRatePerCondition(cond5LeftSuccesses, cond5TrialsLeft)
cond5RightSuccessProportion = successRatePerCondition(cond5RightSuccesses, cond5TrialsRight)

cond6LeftSuccessProportion = successRatePerCondition(cond6LeftSuccesses, cond6TrialsLeft)
cond6RightSuccessProportion = successRatePerCondition(cond6RightSuccesses, cond6TrialsRight)

cond7LeftSuccessProportion = successRatePerCondition(cond7LeftSuccesses, cond7TrialsLeft)
cond7RightSuccessProportion = successRatePerCondition(cond7RightSuccesses, cond7TrialsRight)

In [ ]:
successPropsPerCond = (cond2LeftSuccessProportion, cond2RightSuccessProportion, cond3LeftSuccessProportion, cond3RightSuccessProportion, cond4LeftSuccessProportion, cond4RightSuccessProportion, 
cond5LeftSuccessProportion, cond5LeftSuccessProportion, cond6LeftSuccessProportion, cond6RightSuccessProportion, cond7LeftSuccessProportion,cond7RightSuccessProportion)


In [ ]:
SuccessesPath = '.../'

In [ ]:
saveSuccessDF(SuccessesPath)

In [ ]:
dataPath = '.../'
resultPath = path

In [ ]:
df = pd.read_csv(dataPath + '.../P._skeleton_4_worldpos.csv')

In [ ]:
noP = pd.read_csv(dataPath + 'ArmMoves/NoPerturbation/matchingResult.txt', sep='\t', header=None)
P125Left = pd.read_csv(dataPath + 'ArmMoves/Perturbation125/Left/matchingResult.txt', sep='\t', header=None)
P125Right = pd.read_csv(dataPath + 'ArmMoves/Perturbation125/Right/matchingResult.txt', sep='\t', header=None)
P150Left = pd.read_csv(dataPath + 'ArmMoves/Perturbation150/Left/matchingResult.txt', sep='\t', header=None)
P150Right = pd.read_csv(dataPath + 'ArmMoves/Perturbation150/Right/matchingResult.txt', sep='\t', header=None)
P175Left = pd.read_csv(dataPath + 'ArmMoves/Perturbation175/Left/matchingResult.txt', sep='\t', header=None)
P175Right = pd.read_csv(dataPath + 'ArmMoves/Perturbation175/Right/matchingResult.txt', sep='\t', header=None)
P200Left = pd.read_csv(dataPath + 'ArmMoves/Perturbation200/Left/matchingResult.txt', sep='\t', header=None)
P200Right = pd.read_csv(dataPath + 'ArmMoves/Perturbation200/Right/matchingResult.txt', sep='\t', header=None)
P225Left = pd.read_csv(dataPath + 'ArmMoves/Perturbation225/Left/matchingResult.txt', sep='\t', header=None)
P225Right = pd.read_csv(dataPath + 'ArmMoves/Perturbation225/Right/matchingResult.txt', sep='\t', header=None)
P250Left = pd.read_csv(dataPath + 'ArmMoves/Perturbation250/Left/matchingResult.txt', sep='\t', header=None)
P250Right = pd.read_csv(dataPath + 'ArmMoves/Perturbation250/Right/matchingResult.txt', sep='\t', header=None)


In [ ]:
def getTrajectories(cond = noP, folder = 'NoPerturbation'):
    Moves = getMovements(cond) 
    ArmMoves = getArmMoves(Moves)
    radiusEnd = getJoint(ArmMoves, 'right_radiusEnd.X', 'right_radiusEnd.Z')
    plotJointTrajectory(radiusEnd, TrajectoryFolder = path + 'Trajectories/'+folder+'/RadiusEnd/')

In [ ]:
getTrajectories(noP, 'NoPerturbation')

In [ ]:
radiusEnd = removeBadTrajectory(radiusEnd, [0,1,4])

In [ ]:
DoAnalysis(cond = radiusEnd, folder = 'NoPerturbation')

In [ ]:
Medians = getMedianDuration(radiusEnd)

latent, latentTime, latentStd = getLatent(Medians)
Warps = doDTW(radiusEnd, WarpFolder = path + 'WarpResults/NoPerturbation/RadiusEnd/')

InterpolatedWarps = interpolateWarp(Warps, InterpolResFolder = path + 'InterpolResults/NoPerturbation/RadiusEnd/')

MeanTrajectories = plotMeanVar(InterpolatedWarps, MeanTrajFolder = path + 'MeanTrajectories/NoPerturbation/RadiusEnd/')

Mean3D = plotMean3D(MeanTrajFolder = path + 'MeanTrajectories/NoPerturbation/RadiusEnd/')


In [ ]:
moveDurations = pd.read_csv('.../meanMoveDurations.csv', header=None)

In [ ]:
cond1VelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/NoPerturbation/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/NoPerturbation/Controller/', T = np.linspace(0,moveDurations[1][0]))

cond2LeftVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation125/Left/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation125/Left/Controller/', T = np.linspace(0,moveDurations[1][1]))
cond2RightVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation125/Right/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation125/Right/Controller/', T = np.linspace(0,moveDurations[1][2]))

cond3LeftVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation150/Left/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation150/Left/Controller/', T = np.linspace(0,moveDurations[1][3]))
cond3RightVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation150/Right/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation150/Right/Controller/', T = np.linspace(0,moveDurations[1][4]))

cond4LeftVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation175/Left/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation175/Left/Controller/', T = np.linspace(0,moveDurations[1][5]))
cond4RightVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation175/Right/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation175/Right/Controller/', T = np.linspace(0,moveDurations[1][6]))

cond5LeftVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation200/Left/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation200/Left/Controller/', T = np.linspace(0,moveDurations[1][7]))
cond5RightVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation200/Right/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation200/Right/Controller/', T = np.linspace(0,moveDurations[1][8]))

cond6LeftVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation225/Left/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation225/Left/Controller/', T = np.linspace(0,moveDurations[1][9]))
cond6RightVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation225/Right/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation225/Right/Controller/', T = np.linspace(0,moveDurations[1][10]))

cond7LeftVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation250/Left/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation250/Left/Controller/', T = np.linspace(0,moveDurations[1][11]))
cond7RightVelAcc = getVelAcc(pd.read_csv(path + '/MeanTrajectories/Perturbation250/Right/Controller/Mean.csv', header=None), meanTrajFolder = path + 'MeanTrajectories/Perturbation250/Right/Controller/', T = np.linspace(0,moveDurations[1][12]))

In [ ]:
class Solution:
   def solve(self, L1, L2):
      L1.sort()
      L2.sort()
      ans = float("inf")
      i = j = 0
      while i < len(L1) and j < len(L2):
         ans = min(ans, abs(L1[i] - L2[j]))
         if L1[i] < L2[j]:
            i += 1
         else:
            j += 1
      return ans

In [ ]:
def checkConsistency(Block = pd.read_csv('.csv', sep = ';')):
    test = preprocessing(Block)

    listA = list(test['ViveTracker_x'])

    # using combinations() and lambda
    res = max(combinations(listA, 2), key = lambda sub: abs(sub[0]-sub[1]))

    # print result
    print("Pairs with maximum difference are : ",res)
    print(res[0]-res[1])


    listB = list(test['linkable40All_x'])

    # using combinations() and lambda
    res2 = max(combinations(listB, 2), key = lambda sub: abs(sub[0]-sub[1]))

    # print result
    print("Pairs with maximum difference are : ",res2)
    print(res2[1]-res2[0])

    findVal = np.linspace(0.03,0.05, 21)

    arrayA = np.asarray(listA)
    arrayB = np.asarray(listB)

    for i in findVal:
        print(i)
        print(np.allclose(arrayA, arrayB, rtol=i))

    ob = Solution()
    L1 = listA 
    L2 = listB
    print(ob.solve(L1, L2))

    
    

In [ ]:
checkConsistency(Block = pd.read_csv('.csv', sep = ';'))